# Здесь построим программу для генерации пасспортов скважин

In [7]:
import pandas as pd
import numpy as np
import os

import openpyxl
from openpyxl.cell import WriteOnlyCell
from openpyxl.comments import Comment
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font

from openpyxl.chart import (
    ScatterChart,
    Reference,
    Series,
    LineChart,
)
from openpyxl.chart.axis import DateAxis
from openpyxl.chart.text import RichText
from openpyxl.drawing.text import Paragraph, ParagraphProperties, CharacterProperties, RegularTextRun
from openpyxl.drawing.image import Image

import matplotlib.pyplot as plt

from PIL import Image as PIL_Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

def rename_df(df):
    # переименовываем и удаляем ненужные колонки
    df.rename(columns = {'Unnamed: 1' : 'Позиция по ГП', 'Unnamed: 3':'Скважина', 'Дата.1' : 'ДАТА', 'Unnamed: 13' : 'Высота ТС от уровня земли, м', 
                         'Unnamed: 9' : '№ Термокосы', 'Unnamed: 20' : 'Температура воздуха, °С',
                     0:str('0.0'), 0.5:str('0.5'), 1:str('1.0'), 1.5:str('1.5'), 2:str('2.0'), 2.5:str('2.5'), 3 : str('3.0'), 
                     4:str('4.0'), 5:str('5.0'), 6:str('6.0'), 7:str('7.0'), 8:str('8.0'), 9:str('9.0'), 
                     10:str('10.0'), 12:str('12.0'), 14:str('14.0'), 15:str('15.0')}, inplace = True)
    
    actual_columns = ['Позиция по ГП', 'Скважина', 'ДАТА', 'Высота ТС от уровня земли, м', '0.0', '№ Термокосы', 'Температура воздуха, °С', '0.5', '1.0', '1.5', '2.0', '2.5', '3.0', 
                     '4.0', '5.0', '6.0', '7.0', '8.0', '9.0', 
                     '10.0', '12.0', '14.0', '15.0']
    
    current_col = list(df.columns)
    for temp in current_col:
        dropped_col = []
        if temp not in actual_columns:
            dropped_col.append(temp)
        df.drop(columns = dropped_col, inplace = True)
    df.rename(columns = {'ДАТА' : 'Дата'}, inplace = True)
        
def get_formatting_cells(formating_cells_col, formating_cells_row):
    formating_cells = []
    formating_cells_row.append(formating_cells_row[-1])# правка
    for ind2 in formating_cells_col:
        for ind3 in formating_cells_row:
            formating_cells.append(ind2 + str(ind3))
    return (formating_cells)

def standart_formating_cell(formating_cell):
    formating_cell.alignment= Alignment(horizontal='center', vertical='center', text_rotation=0,
                                        wrap_text=True, shrink_to_fit=False, indent=0)
    formating_cell.font = Font(name='Arial', size=11, bold=False, italic=False,
                               vertAlign=None, underline='none', strike=False, color='FF000000')
    
def plot_graph(df_of_well, name_of_picture_file, list_of_plottiong_columns):
    
    df_plotting = df_of_well[list_of_plottiong_columns]

    df_plotting['Дата'] = df_plotting['Дата'].dt.strftime("%d.%m.%Y") # временно, снять выделение
    
    # размеры шрифтов
    size_of_label_text = 11
    size_of_axis_text = size_of_label_text - 2
    size_of_legend_text=size_of_label_text - 2
    
    fig, ax = plt.subplots(figsize=(11, 3))

    names = df_plotting.columns[1:].astype(float)
    for index, row in df_plotting.iterrows():

        ax.scatter(row[1:], names, s=20)
        ax.plot(row[1:], names, label=row[0])

#     names = list(df_plotting.columns[6:])
    
    ax.legend(loc="lower center", ncol=len(df_plotting), bbox_to_anchor=(0.5,-0.15), frameon=False, 
              fontsize = size_of_legend_text)
    temperatue_ticks = list((np.arange(min(list(df_plotting.dropna(axis = 1).min()[1:]))//1, 
                                       max(list(df_plotting.dropna(axis = 1).max()[1:])), 1))//1+1)
    depth_ticks = list(list((np.arange(0, max(names)+1, 1))))
  
    ax.set_ylabel('Глубина, м', size = size_of_label_text)
    ax.yaxis.set_ticks(depth_ticks)

#     # ax.set_title('Глубина, м')

    ax.set_xlabel('Температура грунта, °С', size = size_of_label_text, labelpad =10)
    ax.xaxis.tick_top()
    ax.xaxis.set_label_position('top')
    ax.xaxis.set_ticks(temperatue_ticks)
#     ax.tick_params(axis='both', which='major', labelsize=size_of_axis_text)

    ax.grid()
    plt.xlim(min(list(df_plotting.dropna(axis = 1).min()[1:]))//1, 
             max(list(df_plotting.dropna(axis = 1).max()[1:]))//1+1)
    plt.ylim(max(list(map(float, (list_of_plottiong_columns[1:])))), 0)
#     plt.show()
    fig.savefig(name_of_picture_file, dpi = 100, bbox_inches = 'tight')
    print('save_pic', well)
    plt.close() # отключаем показ графиков

def get_temperature_mean():
    my_list = list([row['3.0'], row['4.0'], row['5.0'], row['6.0'], row['7.0'], row['8.0'], row['9.0'], row['10.0']])
    my_list = [x for x in my_list if str(x) != 'nan']
    my_list = [x for x in my_list if str(x) != '-']

    if len(my_list) != 0:
        t_mean = round(sum(my_list)/len(my_list),2)
    else:
        t_mean = ''
    return(t_mean)

 
def scale_image(input_image_path,
                output_image_path,
                width=None,
                height=None
                ):
    original_image = PIL_Image.open(input_image_path)
    w, h = original_image.size
#     print('The original image size is {wide} wide x {height} '
#           'high'.format(wide=w, height=h))
 
    if width and height:
        max_size = (width, height)
    elif width:
        max_size = (width, h)
    elif height:
        max_size = (w, height)
    else:
        # No width or height specified
        raise RuntimeError('Width or height required!')
    if w > width:
         original_image.thumbnail(max_size, PIL_Image.ANTIALIAS)
    elif w < width:
#         print('хуй')
        modificator = w/width
        h2 = int(round(h/modificator, 2))
        original_image = original_image.resize((width, h2), PIL_Image.ANTIALIAS)
    original_image.save(output_image_path, optimize=True, quality=0.8)

width_for_10m_well = 960
width_for_15m_well = 1100

## 1. ПСП

In [19]:
path_to_file = 'Том 8/Том VIII Приложение А,Б_ПСП.xlsx' # относительный путь до приложения
list_of_table_names = ['ПСП Весна 2018', 'ПСП Осень 2018', 'ПСП Весна 2019', 'ПСП Осень 2019', 'ПСП Весна 2020', 'ПСП Осень 2020', 'ПСП Весна 2021'] #обрабатываемые листы в порядке заполнения пасспорта

# path_to_file = 'Том 8/Том VIII Приложение В,Г_ГПЭС.xlsx'
# list_of_table_names = ['ГПЭС Весна 2020', 'ГПЭС Осень 2020', 'ГПЭС Весна 2021']

for i, sheet in enumerate (list_of_table_names):
    if i==0:
        df = pd.read_excel(path_to_file, sheet_name = sheet, skiprows = 12, date_parser = lambda x: pd.datetime.strptime(x, "%d.%m.%Y")) # вроде вставка дат работает как надо
        rename_df(df)
    else:
        df_temp = pd.read_excel(path_to_file, sheet_name = sheet, skiprows = 12, date_parser = lambda x: pd.datetime.strptime(x, "%d.%m.%Y")) # вроде вставка дат работает как надо
        rename_df(df_temp)
        df = pd.concat([df, df_temp])
    df = df[df['Дата'].notna()]
######


colour_list = ['00800080', '00008080', '00C0C0C0', '00808080', '009999FF', '00CCFFFF', '00660066', '00FF8080', '000066CC', '00CCCCFF', '00000080', '00FF00FF', '00FFFF00']


name_of_passport_file = path_to_file[:-5]+'_passports'+ path_to_file[-5:]
wb = openpyxl.Workbook()
list_of_unique_wells = list(df['Скважина'].dropna().unique())
# list_of_unique_wells = list_of_unique_wells[180:190] ##### временно
for well in list_of_unique_wells:
    df_of_well = df[df['Скважина'] == well] # пока обойдемся без сортировки

    ### Паспорт термометрической скважины:
    ws = wb.create_sheet(title = well)
    cell = WriteOnlyCell(ws, value="Паспорт термометрической скважины:")
    cell.font = Font(name='Arial', size=14, bold = 'True')
    cell.alignment = Alignment(horizontal = 'right')

    cell_well = WriteOnlyCell(ws, value=well)
    cell_well.font = Font(name='Arial', size=12, bold = 'True')
    cell_well.alignment = Alignment(horizontal = 'center')
    cell_well.border = Border(bottom=Side(border_style='thin', color='FF000000'),)
    cell_Border = WriteOnlyCell(ws, value='')
    cell_Border.border = Border(bottom=Side(border_style='thin', color='FF000000'),)

    ws.append(['']) #первый отступ
    ws.append([cell, '', '', '', '', '', '', '', '', '', '', '', '', cell_well, cell_Border])
    ws.merged_cells.ranges.append("A2:M2")
    ws.merged_cells.ranges.append("N2:O2")

    ### Местоположение термоскважины
    ws.append([])

    cell = WriteOnlyCell(ws, value="Местоположение термоскважины:")
    cell.font = Font(name='Arial', size=12, bold = 'True')
    cell.alignment = Alignment(horizontal = 'right')

    name_of_area = 'Площадка ПСП'
    cell_area = WriteOnlyCell(ws, value=name_of_area)
    cell_area.font = Font(name='Arial', size=12)
    cell_area.alignment = Alignment(horizontal = 'left')

    ws.append([cell, '', '', '', '', '', '', cell_area])
    ws.merged_cells.ranges.append("A4:G4")
    ws.merged_cells.ranges.append("H4:R4")

    cell = WriteOnlyCell(ws, value=df_of_well['Позиция по ГП'].iloc[-1])
    cell.font = Font(name='Arial', size=12)
    cell.alignment = Alignment(horizontal = 'left')
    ws.append(['', '', '', '', '', '', '', cell])
    ws.merged_cells.ranges.append("H5:R5")

    ### формируем таблицу
    ws.append([])

    # условие для 15м скважин
    if list(df_of_well['12.0'].dropna()) or list(df_of_well['14.0'].dropna()) or (list(df_of_well['15.0'].dropna())):
        print('15 м скважина')
        cell = WriteOnlyCell(ws, value="Результаты измерений температуры:")
        cell.font = Font(name='Arial', size=12, bold = 'True')
        cell.alignment = Alignment(horizontal = 'center')

        ws.append([cell])
        ws.merged_cells.ranges.append("A7:R7")      
    
        ws.append(['Дата замера', '№ Термокосы', 'Высота оголовка, м', 'Температура воздуха, °С', 'Температура в скважине на глубине от поверхности земли, м, °С',
                   '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'Температура средняя, °С'])
        ws.append(['', '', '', '', '0.0', '0.5', '1.0', '1.5', '2.0', '2.5', '3.0', '4.0', '5.0', '6.0', '7.0', 
                   '8.0', '9.0', '10.0', '12.0', '14.0', '15.0'])
        
        for index, row in df_of_well.iterrows():
            if row['Дата'] !='-':
                date_value = row['Дата'].strftime("%d.%m.%Y")
            else:
                date_value = row['Дата']
            ws.append([date_value, str(row['№ Термокосы']).replace('.0', ''), row['Высота ТС от уровня земли, м'], row['Температура воздуха, °С'],
                       row['0.0'], row['0.5'], row['1.0'], row['1.5'], row['2.0'], row['2.5'], row['3.0'], row['4.0'], 
                       row['5.0'], row['6.0'], row['7.0'], row['8.0'], row['9.0'], row['10.0'], 
                       row['12.0'], row['14.0'], row['15.0'], get_temperature_mean()])

        ws.merged_cells.ranges.append("A8:A9")
        ws.merged_cells.ranges.append("B8:B9")
        ws.merged_cells.ranges.append("C8:C9")
        ws.merged_cells.ranges.append("D8:D9")
        ws.merged_cells.ranges.append("V8:V9")        
        ws.merged_cells.ranges.append("E8:U8")
        ######################
        #формируем таблицу
        thin = Side(border_style="thin", color="000000")
        double = Side(border_style="double", color="000000")
        medium = Side(border_style="medium", color="000000")
        
        formating_cells = ['A8']
        for formating_cell_position in formating_cells:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(top=medium, left=medium, right=thin, bottom=thin)
            standart_formating_cell(formating_cell)
            
        formating_cells = ['B8','C8', 'D8', 'E8','F8', 'G8', 'H8','I8', 'J8', 'K8','L8', 'M8',
                           'N8','O8', 'P8', 'Q8', 'R8', 'S8', 'T8','U8',]
        for formating_cell_position in formating_cells:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(top=medium, left=thin, right=thin, bottom=thin)
            standart_formating_cell(formating_cell)
            
        formating_cells = ['V8']
        for formating_cell_position in formating_cells:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(top=medium, left=thin, right=medium, bottom=thin)
            standart_formating_cell(formating_cell)
           
        formating_cells = ['A9']
        for formating_cell_position in formating_cells:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(top=thin, left=medium, right=thin, bottom=thin)

            formating_cells = ['B9','C9', 'D9', 'E9','F9', 'G9', 'H9','I9', 'J9', 'K9','L9', 'M9',
                           'N9','O9', 'P9', 'Q9','R9', 'S9', 'T9','U9',]
        for formating_cell_position in formating_cells:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
            standart_formating_cell(formating_cell)
        
        formating_cells = ['V9']
        for formating_cell_position in formating_cells:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(top=thin, left=thin, right=medium, bottom=thin)
            standart_formating_cell(formating_cell)
        
        count_of_meathures = len(df_of_well)
        formating_cells_col = ['A']
        formating_cells_row = list(range(10, 10+count_of_meathures))
        formating_cells = get_formatting_cells(formating_cells_col, formating_cells_row)
        for ind, formating_cell_position in enumerate (formating_cells):
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(left=medium, right=thin)
            standart_formating_cell(formating_cell)
            if str(formating_cells_row[-1]) in formating_cell_position:
                formating_cell = ws[formating_cell_position]
                formating_cell.border = Border(left=medium, right=thin, bottom=medium)
                standart_formating_cell(formating_cell)
        
        count_of_meathures = len(df_of_well)
        formating_cells_col = ['B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
                               'O', 'P', 'Q', 'R' , 'S', 'T', 'U']
        formating_cells_row = list(range(10, 10+count_of_meathures))
        formating_cells = get_formatting_cells(formating_cells_col, formating_cells_row)
        for ind, formating_cell_position in enumerate (formating_cells):
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(left=thin, right=thin)
            standart_formating_cell(formating_cell)
            if str(formating_cells_row[-1]) in formating_cell_position:
                formating_cell = ws[formating_cell_position]
                formating_cell.border = Border(left=thin, right=thin, bottom=medium)
                standart_formating_cell(formating_cell)
                
                
        formating_cells_col = ['V']
        formating_cells_row = list(range(10, 10+count_of_meathures))
        formating_cells = get_formatting_cells(formating_cells_col, formating_cells_row)
        for ind, formating_cell_position in enumerate (formating_cells):
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(left=thin, right=medium)
            standart_formating_cell(formating_cell)
            if str(formating_cells_row[-1]) in formating_cell_position:
                formating_cell = ws[formating_cell_position]
                formating_cell.border = Border(left=thin, right=medium, bottom=medium)
                standart_formating_cell(formating_cell)
        
        # Настраиваем ширину ячеек
        ws.column_dimensions["A"].width = 11
        ws.column_dimensions["B"].width = 7
        ws.column_dimensions["C"].width = 10
        ws.column_dimensions["D"].width = 10
        formating_cells_col = ['E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
                               'O', 'P', 'Q', 'R', 'S', 'T', 'U']
        for column_of_cells in formating_cells_col:
            ws.column_dimensions[column_of_cells].width = 6
        ws.column_dimensions["V"].width = 10

        
        ws.row_dimensions[8].height = 32.531
        ws.row_dimensions[9].height = 32.531
        
        formating_cells_row = list(range(10, 10 + count_of_meathures))
        for form_row in formating_cells_row:
            ws.row_dimensions[form_row].height = 16
        ##################
        ###
        
        ############
#         ВРЕМЕННО, ЗАТЕМ ВКЛЮЧИТЬ
        name_of_picture_file = path_to_file[:-5]+'.files/Graphs/'+str(well)+'.png'
        
        if os.path.exists(name_of_picture_file) == False:
            list_of_plottiong_columns = ['Дата', '0.0', '0.5', '1.0', '1.5', '2.0', '2.5', '3.0', '4.0', '5.0', '6.0', 
                                     '7.0', '8.0', '9.0', '10.0', '12.0', '14.0', '15.0']
            plot_graph(df_of_well, name_of_picture_file, list_of_plottiong_columns)
        scale_image(input_image_path=name_of_picture_file,
                    output_image_path=name_of_picture_file,
                    width=width_for_15m_well,)
        img = Image(name_of_picture_file)
        ws.add_image(img, 'A24')
        #################
        name_of_location_pic_file = path_to_file[:-5]+'.files/Locate/'+str(well).replace('Т', '')+'.png'
        if os.path.exists(name_of_location_pic_file):
            scale_image(input_image_path=name_of_location_pic_file,
                        output_image_path=name_of_location_pic_file,
                        width=width_for_15m_well,)
            img = Image(name_of_location_pic_file)
            ws.add_image(img, 'A45')
        else:
            print('Для скважины', well, 'отсутствует картинка местоположения')

    else:
        # если скважина 10 М
        cell = WriteOnlyCell(ws, value="Результаты измерений температуры:")
        cell.font = Font(name='Arial', size=12, bold = 'True')
        cell.alignment = Alignment(horizontal = 'center')

        ws.append([cell])
        ws.merged_cells.ranges.append("A7:R7")      
    
        ws.append(['Дата замера', '№ Термокосы', 'Высота оголовка, м', 'Температура воздуха, °С', 'Температура в скважине на глубине от поверхности земли, м, °С',
                   '', '', '', '', '', '', '', '', '', '', '', '', '', 'Температура средняя, °С'])
        ws.append(['', '', '', '', '0.0', '0.5', '1.0', '1.5', '2.0', '2.5', '3.0', '4.0', '5.0', '6.0', '7.0', '8.0', '9.0', '10.0'])
        
        for index, row in df_of_well.iterrows():
            if row['Дата'] !='-':
                date_value = row['Дата'].strftime("%d.%m.%Y")
            else:
                date_value = row['Дата']
            ws.append([date_value, str(row['№ Термокосы']).replace('.0', ''), row['Высота ТС от уровня земли, м'], row['Температура воздуха, °С'],
                       row['0.0'], row['0.5'], row['1.0'], row['1.5'], row['2.0'], row['2.5'], row['3.0'], row['4.0'], row['5.0'], row['6.0'], row['7.0'], row['8.0'], row['9.0'], row['10.0'], get_temperature_mean()])

        ws.merged_cells.ranges.append("A8:A9")
        ws.merged_cells.ranges.append("B8:B9")
        ws.merged_cells.ranges.append("C8:C9")
        ws.merged_cells.ranges.append("D8:D9")
        ws.merged_cells.ranges.append("S8:S9")        
        ws.merged_cells.ranges.append("E8:R8")
        ######################
        #формируем таблицу
        thin = Side(border_style="thin", color="000000")
        double = Side(border_style="double", color="000000")
        medium = Side(border_style="medium", color="000000")
        
        formating_cells = ['A8']
        for formating_cell_position in formating_cells:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(top=medium, left=medium, right=thin, bottom=thin)
            standart_formating_cell(formating_cell)
            
        formating_cells = ['B8','C8', 'D8', 'E8','F8', 'G8', 'H8','I8', 'J8', 'K8','L8', 'M8',
                           'N8','O8', 'P8', 'R8','Q8',]
        for formating_cell_position in formating_cells:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(top=medium, left=thin, right=thin, bottom=thin)
            standart_formating_cell(formating_cell)
            
        formating_cells = ['S8']
        for formating_cell_position in formating_cells:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(top=medium, left=thin, right=medium, bottom=thin)
            standart_formating_cell(formating_cell)
           
        formating_cells = ['A9']
        for formating_cell_position in formating_cells:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(top=thin, left=medium, right=thin, bottom=thin)

            formating_cells = ['B9','C9', 'D9', 'E9','F9', 'G9', 'H9','I9', 'J9', 'K9','L9', 'M9',
                           'N9','O9', 'P9', 'Q9','R9',]
        for formating_cell_position in formating_cells:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
            standart_formating_cell(formating_cell)
        
        formating_cells = ['S9']
        for formating_cell_position in formating_cells:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(top=thin, left=thin, right=medium, bottom=thin)
            standart_formating_cell(formating_cell)
        
        count_of_meathures = len(df_of_well)
        formating_cells_col = ['A']
        formating_cells_row = list(range(10, 10+count_of_meathures))
        formating_cells = get_formatting_cells(formating_cells_col, formating_cells_row)
        for ind, formating_cell_position in enumerate (formating_cells):
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(left=medium, right=thin)
            standart_formating_cell(formating_cell)
            if str(formating_cells_row[-1]) in formating_cell_position:
                formating_cell = ws[formating_cell_position]
                formating_cell.border = Border(left=medium, right=thin, bottom=medium)
                standart_formating_cell(formating_cell)
        
        count_of_meathures = len(df_of_well)
        formating_cells_col = ['B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
                               'O', 'P', 'Q', 'R']
        formating_cells_row = list(range(10, 10+count_of_meathures))
        formating_cells = get_formatting_cells(formating_cells_col, formating_cells_row)
        for ind, formating_cell_position in enumerate (formating_cells):
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(left=thin, right=thin)
            standart_formating_cell(formating_cell)
            if str(formating_cells_row[-1]) in formating_cell_position:
                formating_cell = ws[formating_cell_position]
                formating_cell.border = Border(left=thin, right=thin, bottom=medium)
                standart_formating_cell(formating_cell)
                
                
        formating_cells_col = ['S']
        formating_cells_row = list(range(10, 10+count_of_meathures))
        formating_cells = get_formatting_cells(formating_cells_col, formating_cells_row)
        for ind, formating_cell_position in enumerate (formating_cells):
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(left=thin, right=medium)
            standart_formating_cell(formating_cell)
            if str(formating_cells_row[-1]) in formating_cell_position:
                formating_cell = ws[formating_cell_position]
                formating_cell.border = Border(left=thin, right=medium, bottom=medium)
                standart_formating_cell(formating_cell)
        
        # Настраиваем ширину ячеек
        ws.column_dimensions["A"].width = 11
        ws.column_dimensions["B"].width = 7
        ws.column_dimensions["C"].width = 10
        ws.column_dimensions["D"].width = 10
        formating_cells_col = ['E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
                               'O', 'P', 'Q', 'R']
        for column_of_cells in formating_cells_col:
            ws.column_dimensions[column_of_cells].width = 6
        ws.column_dimensions["S"].width = 10

        
        ws.row_dimensions[8].height = 32.531
        ws.row_dimensions[9].height = 32.531
        
        formating_cells_row = list(range(10, 10 + count_of_meathures))
        for form_row in formating_cells_row:
            ws.row_dimensions[form_row].height = 16
        ##################
        ###
        
        ############
#         ВРЕМЕННО, ЗАТЕМ ВКЛЮЧИТЬ
        name_of_picture_file = path_to_file[:-5]+'.files/Graphs/'+str(well)+'.png'
        
        if os.path.exists(name_of_picture_file) == False:
            list_of_plottiong_columns = ['Дата', '0.0', '0.5', '1.0', '1.5', '2.0', '2.5', '3.0', '4.0', '5.0', '6.0', 
                                     '7.0', '8.0', '9.0', '10.0']
            plot_graph(df_of_well, name_of_picture_file, list_of_plottiong_columns)
        scale_image(input_image_path=name_of_picture_file,
                    output_image_path=name_of_picture_file,
                    width=width_for_10m_well,)
        img = Image(name_of_picture_file)
        ws.add_image(img, 'A24')
        #################
        name_of_location_pic_file = path_to_file[:-5]+'.files/Locate/'+str(well).replace('Т', '')+'.png'
        if os.path.exists(name_of_location_pic_file):
            scale_image(input_image_path=name_of_location_pic_file,
                        output_image_path=name_of_location_pic_file,
                        width=width_for_10m_well,)
            img = Image(name_of_location_pic_file)
            ws.add_image(img, 'A45')
        else:
            print('Для скважины', well, 'отсутствует картинка местоположения')
        
wb.save(name_of_passport_file)

print('Vse')


15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
Для скважины ТТ258 отсутствует картинка местоположения
15 м скважина
15 м скважина
15 м скважина
Vse


## 2.ГПЭС

In [20]:
path_to_file = 'Том 8/Том VIII Приложение В,Г_ГПЭС.xlsx'
list_of_table_names = ['ГПЭС Весна 2020', 'ГПЭС Осень 2020', 'ГПЭС Весна 2021']

def rename_df(df):
    # переименовываем и удаляем ненужные колонки
    df.rename(columns = {'Unnamed: 1' : 'Позиция по ГП', 'Unnamed: 3':'Скважина', 'Дата.1' : 'ДАТА', 'Unnamed: 13' : 'Высота ТС от уровня земли, м', 
                         'Unnamed: 9' : '№ Термокосы', 'Unnamed: 20' : 'Температура воздуха, °С',
                     0:str('0.0'), 0.5:str('0.5'), 1:str('1.0'), 1.5:str('1.5'), 2:str('2.0'), 2.5:str('2.5'), 3 : str('3.0'), 
                     4:str('4.0'), 5:str('5.0'), 6:str('6.0'), 7:str('7.0'), 8:str('8.0'), 9:str('9.0'), 
                     10:str('10.0'), 12:str('12.0'), 14:str('14.0'), 15:str('15.0')}, inplace = True)
    
    actual_columns = ['Позиция по ГП', 'Скважина', 'ДАТА', 'Высота ТС от уровня земли, м', '0.0', '№ Термокосы', 'Температура воздуха, °С', '0.5', '1.0', '1.5', '2.0', '2.5', '3.0', 
                     '4.0', '5.0', '6.0', '7.0', '8.0', '9.0', 
                     '10.0', '12.0', '14.0', '15.0']
    
    current_col = list(df.columns)
    for temp in current_col:
        dropped_col = []
        if temp not in actual_columns:
            dropped_col.append(temp)
        df.drop(columns = dropped_col, inplace = True)
    df.rename(columns = {'ДАТА' : 'Дата'}, inplace = True)

for i, sheet in enumerate (list_of_table_names):
    if i==0:
        df = pd.read_excel(path_to_file, sheet_name = sheet, skiprows = 12,) # вроде вставка дат работает как надо
        rename_df(df)
    else:
        df_temp = pd.read_excel(path_to_file, sheet_name = sheet, skiprows = 12,) # вроде вставка дат работает как надо
        rename_df(df_temp)
        df = pd.concat([df, df_temp])
    
    df = df[df['Дата'].notna()]
######


colour_list = ['00800080', '00008080', '00C0C0C0', '00808080', '009999FF', '00CCFFFF', '00660066', '00FF8080', '000066CC', '00CCCCFF', '00000080', '00FF00FF', '00FFFF00']


name_of_passport_file = path_to_file[:-5]+'_passports'+ path_to_file[-5:]
wb = openpyxl.Workbook()
list_of_unique_wells = list(df['Скважина'].dropna().unique())
# list_of_unique_wells = list_of_unique_wells[180:190] ##### временно
for well in list_of_unique_wells:
    df_of_well = df[df['Скважина'] == well] # пока обойдемся без сортировки

    ### Паспорт термометрической скважины:
    ws = wb.create_sheet(title = str(well))
    cell = WriteOnlyCell(ws, value="Паспорт термометрической скважины:")
    cell.font = Font(name='Arial', size=14, bold = 'True')
    cell.alignment = Alignment(horizontal = 'right')

    cell_well = WriteOnlyCell(ws, value=well)
    cell_well.font = Font(name='Arial', size=12, bold = 'True')
    cell_well.alignment = Alignment(horizontal = 'center')
    cell_well.border = Border(bottom=Side(border_style='thin', color='FF000000'),)
    cell_Border = WriteOnlyCell(ws, value='')
    cell_Border.border = Border(bottom=Side(border_style='thin', color='FF000000'),)

    ws.append(['']) #первый отступ
    ws.append([cell, '', '', '', '', '', '', '', '', '', '', '', '', cell_well, cell_Border])
    ws.merged_cells.ranges.append("A2:M2")
    ws.merged_cells.ranges.append("N2:O2")

    ### Местоположение термоскважины
    ws.append([])

    cell = WriteOnlyCell(ws, value="Местоположение термоскважины:")
    cell.font = Font(name='Arial', size=12, bold = 'True')
    cell.alignment = Alignment(horizontal = 'right')

    name_of_area = 'Площадка ПСП'
    cell_area = WriteOnlyCell(ws, value=name_of_area)
    cell_area.font = Font(name='Arial', size=12)
    cell_area.alignment = Alignment(horizontal = 'left')

    ws.append([cell, '', '', '', '', '', '', cell_area])
    ws.merged_cells.ranges.append("A4:G4")
    ws.merged_cells.ranges.append("H4:R4")

    cell = WriteOnlyCell(ws, value=df_of_well['Позиция по ГП'].iloc[-1])
    cell.font = Font(name='Arial', size=12)
    cell.alignment = Alignment(horizontal = 'left')
    ws.append(['', '', '', '', '', '', '', cell])
    ws.merged_cells.ranges.append("H5:R5")

    ### формируем таблицу
    ws.append([])

    # условие для 15м скважин
    
    # если скважина 10 М
    cell = WriteOnlyCell(ws, value="Результаты измерений температуры:")
    cell.font = Font(name='Arial', size=12, bold = 'True')
    cell.alignment = Alignment(horizontal = 'center')

    ws.append([cell])
    ws.merged_cells.ranges.append("A7:R7")      

    ws.append(['Дата замера', '№ Термокосы', 'Высота оголовка, м', 'Температура воздуха, °С', 'Температура в скважине на глубине от поверхности земли, м, °С',
               '', '', '', '', '', '', '', '', '', '', '', '', '', 'Температура средняя, °С'])
    ws.append(['', '', '', '', '0.0', '0.5', '1.0', '1.5', '2.0', '2.5', '3.0', '4.0', '5.0', '6.0', '7.0', '8.0', '9.0', '10.0'])

    for index, row in df_of_well.iterrows():
        if row['Дата'] !='-':
            date_value = row['Дата'].strftime("%d.%m.%Y")
        else:
            date_value = row['Дата']
        ws.append([date_value, str(row['№ Термокосы']).replace('.0', ''), row['Высота ТС от уровня земли, м'], row['Температура воздуха, °С'],
                   row['0.0'], row['0.5'], row['1.0'], row['1.5'], row['2.0'], row['2.5'], row['3.0'], row['4.0'], row['5.0'], row['6.0'], row['7.0'], row['8.0'], row['9.0'], row['10.0'], get_temperature_mean()])

    ws.merged_cells.ranges.append("A8:A9")
    ws.merged_cells.ranges.append("B8:B9")
    ws.merged_cells.ranges.append("C8:C9")
    ws.merged_cells.ranges.append("D8:D9")
    ws.merged_cells.ranges.append("S8:S9")        
    ws.merged_cells.ranges.append("E8:R8")
    ######################
    #формируем таблицу
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    medium = Side(border_style="medium", color="000000")

    formating_cells = ['A8']
    for formating_cell_position in formating_cells:
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(top=medium, left=medium, right=thin, bottom=thin)
        standart_formating_cell(formating_cell)

    formating_cells = ['B8','C8', 'D8', 'E8','F8', 'G8', 'H8','I8', 'J8', 'K8','L8', 'M8',
                       'N8','O8', 'P8', 'R8','Q8',]
    for formating_cell_position in formating_cells:
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(top=medium, left=thin, right=thin, bottom=thin)
        standart_formating_cell(formating_cell)

    formating_cells = ['S8']
    for formating_cell_position in formating_cells:
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(top=medium, left=thin, right=medium, bottom=thin)
        standart_formating_cell(formating_cell)

    formating_cells = ['A9']
    for formating_cell_position in formating_cells:
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(top=thin, left=medium, right=thin, bottom=thin)

        formating_cells = ['B9','C9', 'D9', 'E9','F9', 'G9', 'H9','I9', 'J9', 'K9','L9', 'M9',
                       'N9','O9', 'P9', 'Q9','R9',]
    for formating_cell_position in formating_cells:
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
        standart_formating_cell(formating_cell)

    formating_cells = ['S9']
    for formating_cell_position in formating_cells:
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(top=thin, left=thin, right=medium, bottom=thin)
        standart_formating_cell(formating_cell)

    count_of_meathures = len(df_of_well)
    formating_cells_col = ['A']
    formating_cells_row = list(range(10, 10+count_of_meathures))
    formating_cells = get_formatting_cells(formating_cells_col, formating_cells_row)
    for ind, formating_cell_position in enumerate (formating_cells):
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(left=medium, right=thin)
        standart_formating_cell(formating_cell)
        if str(formating_cells_row[-1]) in formating_cell_position:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(left=medium, right=thin, bottom=medium)
            standart_formating_cell(formating_cell)

    count_of_meathures = len(df_of_well)
    formating_cells_col = ['B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
                           'O', 'P', 'Q', 'R']
    formating_cells_row = list(range(10, 10+count_of_meathures))
    formating_cells = get_formatting_cells(formating_cells_col, formating_cells_row)
    for ind, formating_cell_position in enumerate (formating_cells):
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(left=thin, right=thin)
        standart_formating_cell(formating_cell)
        if str(formating_cells_row[-1]) in formating_cell_position:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(left=thin, right=thin, bottom=medium)
            standart_formating_cell(formating_cell)


    formating_cells_col = ['S']
    formating_cells_row = list(range(10, 10+count_of_meathures))
    formating_cells = get_formatting_cells(formating_cells_col, formating_cells_row)
    for ind, formating_cell_position in enumerate (formating_cells):
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(left=thin, right=medium)
        standart_formating_cell(formating_cell)
        if str(formating_cells_row[-1]) in formating_cell_position:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(left=thin, right=medium, bottom=medium)
            standart_formating_cell(formating_cell)

    # Настраиваем ширину ячеек
    ws.column_dimensions["A"].width = 11
    ws.column_dimensions["B"].width = 7
    ws.column_dimensions["C"].width = 10
    ws.column_dimensions["D"].width = 10
    formating_cells_col = ['E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
                           'O', 'P', 'Q', 'R']
    for column_of_cells in formating_cells_col:
        ws.column_dimensions[column_of_cells].width = 6
    ws.column_dimensions["S"].width = 10


    ws.row_dimensions[8].height = 32.531
    ws.row_dimensions[9].height = 32.531

    formating_cells_row = list(range(10, 10 + count_of_meathures))
    for form_row in formating_cells_row:
        ws.row_dimensions[form_row].height = 16
    ##################
    ###

    ############
#         ВРЕМЕННО, ЗАТЕМ ВКЛЮЧИТЬ
    name_of_picture_file = path_to_file[:-5]+'.files/Graphs/'+str(well)+'.png'

    if os.path.exists(name_of_picture_file) == False:
        list_of_plottiong_columns = ['Дата', '0.0', '0.5', '1.0', '1.5', '2.0', '2.5', '3.0', '4.0', '5.0', '6.0', 
                                 '7.0', '8.0', '9.0', '10.0']
        plot_graph(df_of_well, name_of_picture_file, list_of_plottiong_columns)

    scale_image(input_image_path=name_of_picture_file,
                output_image_path=name_of_picture_file,
                width=width_for_10m_well,)
    img = Image(name_of_picture_file)
    ws.add_image(img, 'A24')
    #################
    name_of_location_pic_file = path_to_file[:-5]+'.files/Locate/'+str(well).replace('Т', '')+'.png'
    if os.path.exists(name_of_location_pic_file):
        scale_image(input_image_path=name_of_location_pic_file,
                    output_image_path=name_of_location_pic_file,
                    width=width_for_10m_well,)
        img = Image(name_of_location_pic_file)
        ws.add_image(img, 'A45')
    else:
        print('Для скважины', well, 'отсутствует картинка местоположения')
#         

wb.save(name_of_passport_file)

print('Vse')

Vse


## 3.ГТЭС

In [94]:
path_to_file = 'Том 8/Том VIII Приложение Д,Е_ГТЭС.xlsx'
list_of_table_names = ['ГТЭС Весна 2018', 'ГТЭС Осень 2018', 'ГТЭС Весна 2019', 'ГТЭС Осень 2019', 'ГТЭС Осень 2020', 'ГТЭС Весна 2021'] #обрабатываемые листы в порядке заполнения пасспорта

def rename_df(df):
    # переименовываем и удаляем ненужные колонки
    df.rename(columns = {'Unnamed: 1' : 'Позиция по ГП', 'Unnamed: 2':'Скважина', 'Дата.1' : 'ДАТА', 'Unnamed: 13' : 'Высота ТС от уровня земли, м', # правка
                         'Unnamed: 9' : '№ Термокосы', 'Unnamed: 20' : 'Температура воздуха, °С',
                     0:str('0.0'), 0.5:str('0.5'), 1:str('1.0'), 1.5:str('1.5'), 2:str('2.0'), 2.5:str('2.5'), 3 : str('3.0'), 
                     4:str('4.0'), 5:str('5.0'), 6:str('6.0'), 7:str('7.0'), 8:str('8.0'), 9:str('9.0'), 
                     10:str('10.0'), 12:str('12.0'), 14:str('14.0'), 15:str('15.0')}, inplace = True)
    
    actual_columns = ['Позиция по ГП', 'Скважина', 'ДАТА', 'Высота ТС от уровня земли, м', '0.0', '№ Термокосы', 'Температура воздуха, °С', '0.5', '1.0', '1.5', '2.0', '2.5', '3.0', 
                     '4.0', '5.0', '6.0', '7.0', '8.0', '9.0', 
                     '10.0', '12.0', '14.0', '15.0']
    
    current_col = list(df.columns)
    for temp in current_col:
        dropped_col = []
        if temp not in actual_columns:
            dropped_col.append(temp)
        df.drop(columns = dropped_col, inplace = True)
    df.rename(columns = {'ДАТА' : 'Дата'}, inplace = True)

for i, sheet in enumerate (list_of_table_names):
    if i==0:
        df = pd.read_excel(path_to_file, sheet_name = sheet, skiprows = 12,) # вроде вставка дат работает как надо
        rename_df(df)
    else:
        df_temp = pd.read_excel(path_to_file, sheet_name = sheet, skiprows = 12,) # вроде вставка дат работает как надо
        rename_df(df_temp)
        df = pd.concat([df, df_temp])
    
    df = df[df['Дата'].notna()]
######


colour_list = ['00800080', '00008080', '00C0C0C0', '00808080', '009999FF', '00CCFFFF', '00660066', '00FF8080', '000066CC', '00CCCCFF', '00000080', '00FF00FF', '00FFFF00']


name_of_passport_file = path_to_file[:-5]+'_passports'+ path_to_file[-5:]
wb = openpyxl.Workbook()
list_of_unique_wells = list(df['Скважина'].dropna().unique())
# list_of_unique_wells = list_of_unique_wells[180:190] ##### временно
for well in list_of_unique_wells:
    df_of_well = df[df['Скважина'] == well] # пока обойдемся без сортировки

    ### Паспорт термометрической скважины:
    ws = wb.create_sheet(title = str(well))
    cell = WriteOnlyCell(ws, value="Паспорт термометрической скважины:")
    cell.font = Font(name='Arial', size=14, bold = 'True')
    cell.alignment = Alignment(horizontal = 'right')

    cell_well = WriteOnlyCell(ws, value=well)
    cell_well.font = Font(name='Arial', size=12, bold = 'True')
    cell_well.alignment = Alignment(horizontal = 'center')
    cell_well.border = Border(bottom=Side(border_style='thin', color='FF000000'),)
    cell_Border = WriteOnlyCell(ws, value='')
    cell_Border.border = Border(bottom=Side(border_style='thin', color='FF000000'),)

    ws.append(['']) #первый отступ
    ws.append([cell, '', '', '', '', '', '', '', '', '', '', '', '', cell_well, cell_Border])
    ws.merged_cells.ranges.append("A2:M2")
    ws.merged_cells.ranges.append("N2:O2")

    ### Местоположение термоскважины
    ws.append([])

    cell = WriteOnlyCell(ws, value="Местоположение термоскважины:")
    cell.font = Font(name='Arial', size=12, bold = 'True')
    cell.alignment = Alignment(horizontal = 'right')

    name_of_area = 'Площадка ПСП'
    cell_area = WriteOnlyCell(ws, value=name_of_area)
    cell_area.font = Font(name='Arial', size=12)
    cell_area.alignment = Alignment(horizontal = 'left')

    ws.append([cell, '', '', '', '', '', '', cell_area])
    ws.merged_cells.ranges.append("A4:G4")
    ws.merged_cells.ranges.append("H4:R4")

    cell = WriteOnlyCell(ws, value=df_of_well['Позиция по ГП'].iloc[-1])
    cell.font = Font(name='Arial', size=12)
    cell.alignment = Alignment(horizontal = 'left')
    ws.append(['', '', '', '', '', '', '', cell])
    ws.merged_cells.ranges.append("H5:R5")

    ### формируем таблицу
    ws.append([])

    # условие для 15м скважин
    
    # если скважина 10 М
    cell = WriteOnlyCell(ws, value="Результаты измерений температуры:")
    cell.font = Font(name='Arial', size=12, bold = 'True')
    cell.alignment = Alignment(horizontal = 'center')

    ws.append([cell])
    ws.merged_cells.ranges.append("A7:R7")      

    ws.append(['Дата замера', '№ Термокосы', 'Высота оголовка, м', 'Температура воздуха, °С', 'Температура в скважине на глубине от поверхности земли, м, °С',
               '', '', '', '', '', '', '', '', '', '', '', '', '', 'Температура средняя, °С'])
    ws.append(['', '', '', '', '0.0', '0.5', '1.0', '1.5', '2.0', '2.5', '3.0', '4.0', '5.0', '6.0', '7.0', '8.0', '9.0', '10.0'])

    for index, row in df_of_well.iterrows():
        if row['Дата'] !='-':
            date_value = row['Дата'].strftime("%d.%m.%Y")
        else:
            date_value = row['Дата']
        ws.append([date_value, str(row['№ Термокосы']).replace('.0', ''), row['Высота ТС от уровня земли, м'], row['Температура воздуха, °С'],
                   row['0.0'], row['0.5'], row['1.0'], row['1.5'], row['2.0'], row['2.5'], row['3.0'], row['4.0'], row['5.0'], row['6.0'], row['7.0'], row['8.0'], row['9.0'], row['10.0'], get_temperature_mean()])

    ws.merged_cells.ranges.append("A8:A9")
    ws.merged_cells.ranges.append("B8:B9")
    ws.merged_cells.ranges.append("C8:C9")
    ws.merged_cells.ranges.append("D8:D9")
    ws.merged_cells.ranges.append("S8:S9")        
    ws.merged_cells.ranges.append("E8:R8")
    ######################
    #формируем таблицу
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    medium = Side(border_style="medium", color="000000")

    formating_cells = ['A8']
    for formating_cell_position in formating_cells:
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(top=medium, left=medium, right=thin, bottom=thin)
        standart_formating_cell(formating_cell)

    formating_cells = ['B8','C8', 'D8', 'E8','F8', 'G8', 'H8','I8', 'J8', 'K8','L8', 'M8',
                       'N8','O8', 'P8', 'R8','Q8',]
    for formating_cell_position in formating_cells:
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(top=medium, left=thin, right=thin, bottom=thin)
        standart_formating_cell(formating_cell)

    formating_cells = ['S8']
    for formating_cell_position in formating_cells:
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(top=medium, left=thin, right=medium, bottom=thin)
        standart_formating_cell(formating_cell)

    formating_cells = ['A9']
    for formating_cell_position in formating_cells:
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(top=thin, left=medium, right=thin, bottom=thin)

        formating_cells = ['B9','C9', 'D9', 'E9','F9', 'G9', 'H9','I9', 'J9', 'K9','L9', 'M9',
                       'N9','O9', 'P9', 'Q9','R9',]
    for formating_cell_position in formating_cells:
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
        standart_formating_cell(formating_cell)

    formating_cells = ['S9']
    for formating_cell_position in formating_cells:
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(top=thin, left=thin, right=medium, bottom=thin)
        standart_formating_cell(formating_cell)

    count_of_meathures = len(df_of_well)
    formating_cells_col = ['A']
    formating_cells_row = list(range(10, 10+count_of_meathures))
    formating_cells = get_formatting_cells(formating_cells_col, formating_cells_row)
    for ind, formating_cell_position in enumerate (formating_cells):
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(left=medium, right=thin)
        standart_formating_cell(formating_cell)
        if str(formating_cells_row[-1]) in formating_cell_position:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(left=medium, right=thin, bottom=medium)
            standart_formating_cell(formating_cell)

    count_of_meathures = len(df_of_well)
    formating_cells_col = ['B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
                           'O', 'P', 'Q', 'R']
    formating_cells_row = list(range(10, 10+count_of_meathures))
    formating_cells = get_formatting_cells(formating_cells_col, formating_cells_row)
    for ind, formating_cell_position in enumerate (formating_cells):
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(left=thin, right=thin)
        standart_formating_cell(formating_cell)
        if str(formating_cells_row[-1]) in formating_cell_position:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(left=thin, right=thin, bottom=medium)
            standart_formating_cell(formating_cell)


    formating_cells_col = ['S']
    formating_cells_row = list(range(10, 10+count_of_meathures))
    formating_cells = get_formatting_cells(formating_cells_col, formating_cells_row)
    for ind, formating_cell_position in enumerate (formating_cells):
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(left=thin, right=medium)
        standart_formating_cell(formating_cell)
        if str(formating_cells_row[-1]) in formating_cell_position:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(left=thin, right=medium, bottom=medium)
            standart_formating_cell(formating_cell)

    # Настраиваем ширину ячеек
    ws.column_dimensions["A"].width = 11
    ws.column_dimensions["B"].width = 7
    ws.column_dimensions["C"].width = 10
    ws.column_dimensions["D"].width = 10
    formating_cells_col = ['E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
                           'O', 'P', 'Q', 'R']
    for column_of_cells in formating_cells_col:
        ws.column_dimensions[column_of_cells].width = 6
    ws.column_dimensions["S"].width = 10


    ws.row_dimensions[8].height = 32.531
    ws.row_dimensions[9].height = 32.531

    formating_cells_row = list(range(10, 10 + count_of_meathures))
    for form_row in formating_cells_row:
        ws.row_dimensions[form_row].height = 16
    ##################
    ###

    ############
#         ВРЕМЕННО, ЗАТЕМ ВКЛЮЧИТЬ
    name_of_picture_file = path_to_file[:-5]+'.files/Graphs/'+str(well)+'.png'

    if os.path.exists(name_of_picture_file) == False:
        list_of_plottiong_columns = ['Дата', '0.0', '0.5', '1.0', '1.5', '2.0', '2.5', '3.0', '4.0', '5.0', '6.0', 
                                 '7.0', '8.0', '9.0', '10.0']
        plot_graph(df_of_well, name_of_picture_file, list_of_plottiong_columns)
    
    scale_image(input_image_path=name_of_picture_file,
                output_image_path=name_of_picture_file,
                width=width_for_10m_well,)
    img = Image(name_of_picture_file)
    ws.add_image(img, 'A24')
    #################
    name_of_location_pic_file = path_to_file[:-5]+'.files/Locate/'+str(well)+'.png'
    
    
    if os.path.exists(name_of_location_pic_file):
        scale_image(input_image_path=name_of_location_pic_file,
                    output_image_path=name_of_location_pic_file,
                    width=width_for_10m_well
        img = Image(name_of_location_pic_file)
        ws.add_image(img, 'A45')
    else:
        print('Для скважины', well, 'отсутствует картинка местоположения')
wb.save(name_of_passport_file)

print('Vse')

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Для скважины 115 отсутствует картинка местоположения
Для скважины 108 отсутствует картинка местоположения
Для скважины 116 отсутствует картинка местоположения
Для скважины ТТ5(2) отсутствует картинка местоположения
Для скважины ТТ8 отсутствует картинка местоположения
Для скважины 133 отсутствует картинка местоположения
Для скважины 132 отсутствует картинка местоположения
Для скважины 118 отсутствует картинка местоположения
Для скважины 131 отсутствует картинка местоположения
Для скважины 134 отсутствует картинка местоположения
Для скважины 167 отсутствует картинка местоположения
Для скважины 129 отсутствует картинка местоположения
Для скважины 96 отсутствует картинка местоположения
Для скважины 161 отсутствует картинка местоположения
Vse


## 4. ППиТБО

In [21]:
path_to_file = 'Том 8/Том VIII Приложение Ж,З_ППиТБО.xlsx' # относительный путь до приложения
list_of_table_names = ['ППиТБО Осень 2019', 'ППиТБО Осень 2020', 'ППиТБО Весна 2021'] #обрабатываемые листы в порядке заполнения пасспорта

def rename_df(df):
    # переименовываем и удаляем ненужные колонки
    df.rename(columns = {'Unnamed: 1' : 'Позиция по ГП', 'Unnamed: 2':'Скважина', 'Дата.1' : 'ДАТА', 'Unnamed: 13' : 'Высота ТС от уровня земли, м', 
                         'Unnamed: 9' : '№ Термокосы', 'Unnamed: 20' : 'Температура воздуха, °С',
                     0:str('0.0'), 0.5:str('0.5'), 1:str('1.0'), 1.5:str('1.5'), 2:str('2.0'), 2.5:str('2.5'), 3 : str('3.0'), 
                     4:str('4.0'), 5:str('5.0'), 6:str('6.0'), 7:str('7.0'), 8:str('8.0'), 9:str('9.0'), 
                     10:str('10.0'), 12:str('12.0'), 14:str('14.0'), 15:str('15.0')}, inplace = True)
    
    actual_columns = ['Позиция по ГП', 'Скважина', 'ДАТА', 'Высота ТС от уровня земли, м', '0.0', '№ Термокосы', 'Температура воздуха, °С', '0.5', '1.0', '1.5', '2.0', '2.5', '3.0', 
                     '4.0', '5.0', '6.0', '7.0', '8.0', '9.0', 
                     '10.0', '12.0', '14.0', '15.0']
    
    current_col = list(df.columns)
    for temp in current_col:
        dropped_col = []
        if temp not in actual_columns:
            dropped_col.append(temp)
        df.drop(columns = dropped_col, inplace = True)
    df.rename(columns = {'ДАТА' : 'Дата'}, inplace = True)


for i, sheet in enumerate (list_of_table_names):
    if i==0:
        df = pd.read_excel(path_to_file, sheet_name = sheet, skiprows = 12, date_parser = lambda x: pd.datetime.strptime(x, "%d.%m.%Y")) # вроде вставка дат работает как надо
        rename_df(df)
    else:
        df_temp = pd.read_excel(path_to_file, sheet_name = sheet, skiprows = 12, date_parser = lambda x: pd.datetime.strptime(x, "%d.%m.%Y")) # вроде вставка дат работает как надо
        rename_df(df_temp)
        df = pd.concat([df, df_temp])
    df = df[df['Дата'].notna()]
######


colour_list = ['00800080', '00008080', '00C0C0C0', '00808080', '009999FF', '00CCFFFF', '00660066', '00FF8080', '000066CC', '00CCCCFF', '00000080', '00FF00FF', '00FFFF00']


name_of_passport_file = path_to_file[:-5]+'_passports'+ path_to_file[-5:]
wb = openpyxl.Workbook()
list_of_unique_wells = list(df['Скважина'].dropna().unique())
# list_of_unique_wells = list_of_unique_wells[180:190] ##### временно
for well in list_of_unique_wells:
    df_of_well = df[df['Скважина'] == well] # пока обойдемся без сортировки

    ### Паспорт термометрической скважины:
    ws = wb.create_sheet(title = str(well).replace('.0', ''))
    cell = WriteOnlyCell(ws, value="Паспорт термометрической скважины:")
    cell.font = Font(name='Arial', size=14, bold = 'True')
    cell.alignment = Alignment(horizontal = 'right')

    cell_well = WriteOnlyCell(ws, value=well)
    cell_well.font = Font(name='Arial', size=12, bold = 'True')
    cell_well.alignment = Alignment(horizontal = 'center')
    cell_well.border = Border(bottom=Side(border_style='thin', color='FF000000'),)
    cell_Border = WriteOnlyCell(ws, value='')
    cell_Border.border = Border(bottom=Side(border_style='thin', color='FF000000'),)

    ws.append(['']) #первый отступ
    ws.append([cell, '', '', '', '', '', '', '', '', '', '', '', '', cell_well, cell_Border])
    ws.merged_cells.ranges.append("A2:M2")
    ws.merged_cells.ranges.append("N2:O2")

    ### Местоположение термоскважины
    ws.append([])

    cell = WriteOnlyCell(ws, value="Местоположение термоскважины:")
    cell.font = Font(name='Arial', size=12, bold = 'True')
    cell.alignment = Alignment(horizontal = 'right')

    name_of_area = 'Площадка ПСП'
    cell_area = WriteOnlyCell(ws, value=name_of_area)
    cell_area.font = Font(name='Arial', size=12)
    cell_area.alignment = Alignment(horizontal = 'left')

    ws.append([cell, '', '', '', '', '', '', cell_area])
    ws.merged_cells.ranges.append("A4:G4")
    ws.merged_cells.ranges.append("H4:R4")

    cell = WriteOnlyCell(ws, value=df_of_well['Позиция по ГП'].iloc[-1])
    cell.font = Font(name='Arial', size=12)
    cell.alignment = Alignment(horizontal = 'left')
    ws.append(['', '', '', '', '', '', '', cell])
    ws.merged_cells.ranges.append("H5:R5")

    ### формируем таблицу
    ws.append([])

    # условие для 15м скважин
    if list(df_of_well['12.0'].dropna()) or list(df_of_well['14.0'].dropna()) or (list(df_of_well['15.0'].dropna())):
        print('15 м скважина')
        cell = WriteOnlyCell(ws, value="Результаты измерений температуры:")
        cell.font = Font(name='Arial', size=12, bold = 'True')
        cell.alignment = Alignment(horizontal = 'center')

        ws.append([cell])
        ws.merged_cells.ranges.append("A7:R7")      
    
        ws.append(['Дата замера', '№ Термокосы', 'Высота оголовка, м', 'Температура воздуха, °С', 'Температура в скважине на глубине от поверхности земли, м, °С',
                   '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'Температура средняя, °С'])
        ws.append(['', '', '', '', '0.0', '0.5', '1.0', '1.5', '2.0', '2.5', '3.0', '4.0', '5.0', '6.0', '7.0', 
                   '8.0', '9.0', '10.0', '12.0', '14.0', '15.0'])
        
        for index, row in df_of_well.iterrows():
            if row['Дата'] !='-':
                date_value = row['Дата'].strftime("%d.%m.%Y")
            else:
                date_value = row['Дата']
            ws.append([date_value, str(row['№ Термокосы']).replace('.0', ''), row['Высота ТС от уровня земли, м'], row['Температура воздуха, °С'],
                       row['0.0'], row['0.5'], row['1.0'], row['1.5'], row['2.0'], row['2.5'], row['3.0'], row['4.0'], 
                       row['5.0'], row['6.0'], row['7.0'], row['8.0'], row['9.0'], row['10.0'], 
                       row['12.0'], row['14.0'], row['15.0'], get_temperature_mean()])

        ws.merged_cells.ranges.append("A8:A9")
        ws.merged_cells.ranges.append("B8:B9")
        ws.merged_cells.ranges.append("C8:C9")
        ws.merged_cells.ranges.append("D8:D9")
        ws.merged_cells.ranges.append("V8:V9")        
        ws.merged_cells.ranges.append("E8:U8")
        ######################
        #формируем таблицу
        thin = Side(border_style="thin", color="000000")
        double = Side(border_style="double", color="000000")
        medium = Side(border_style="medium", color="000000")
        
        formating_cells = ['A8']
        for formating_cell_position in formating_cells:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(top=medium, left=medium, right=thin, bottom=thin)
            standart_formating_cell(formating_cell)
            
        formating_cells = ['B8','C8', 'D8', 'E8','F8', 'G8', 'H8','I8', 'J8', 'K8','L8', 'M8',
                           'N8','O8', 'P8', 'Q8', 'R8', 'S8', 'T8','U8',]
        for formating_cell_position in formating_cells:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(top=medium, left=thin, right=thin, bottom=thin)
            standart_formating_cell(formating_cell)
            
        formating_cells = ['V8']
        for formating_cell_position in formating_cells:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(top=medium, left=thin, right=medium, bottom=thin)
            standart_formating_cell(formating_cell)
           
        formating_cells = ['A9']
        for formating_cell_position in formating_cells:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(top=thin, left=medium, right=thin, bottom=thin)

            formating_cells = ['B9','C9', 'D9', 'E9','F9', 'G9', 'H9','I9', 'J9', 'K9','L9', 'M9',
                           'N9','O9', 'P9', 'Q9','R9', 'S9', 'T9','U9',]
        for formating_cell_position in formating_cells:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
            standart_formating_cell(formating_cell)
        
        formating_cells = ['V9']
        for formating_cell_position in formating_cells:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(top=thin, left=thin, right=medium, bottom=thin)
            standart_formating_cell(formating_cell)
        
        count_of_meathures = len(df_of_well)
        formating_cells_col = ['A']
        formating_cells_row = list(range(10, 10+count_of_meathures))
        formating_cells = get_formatting_cells(formating_cells_col, formating_cells_row)
        for ind, formating_cell_position in enumerate (formating_cells):
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(left=medium, right=thin)
            standart_formating_cell(formating_cell)
            if str(formating_cells_row[-1]) in formating_cell_position:
                formating_cell = ws[formating_cell_position]
                formating_cell.border = Border(left=medium, right=thin, bottom=medium)
                standart_formating_cell(formating_cell)
        
        count_of_meathures = len(df_of_well)
        formating_cells_col = ['B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
                               'O', 'P', 'Q', 'R' , 'S', 'T', 'U']
        formating_cells_row = list(range(10, 10+count_of_meathures))
        formating_cells = get_formatting_cells(formating_cells_col, formating_cells_row)
        for ind, formating_cell_position in enumerate (formating_cells):
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(left=thin, right=thin)
            standart_formating_cell(formating_cell)
            if str(formating_cells_row[-1]) in formating_cell_position:
                formating_cell = ws[formating_cell_position]
                formating_cell.border = Border(left=thin, right=thin, bottom=medium)
                standart_formating_cell(formating_cell)
                
                
        formating_cells_col = ['V']
        formating_cells_row = list(range(10, 10+count_of_meathures))
        formating_cells = get_formatting_cells(formating_cells_col, formating_cells_row)
        for ind, formating_cell_position in enumerate (formating_cells):
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(left=thin, right=medium)
            standart_formating_cell(formating_cell)
            if str(formating_cells_row[-1]) in formating_cell_position:
                formating_cell = ws[formating_cell_position]
                formating_cell.border = Border(left=thin, right=medium, bottom=medium)
                standart_formating_cell(formating_cell)
        
        # Настраиваем ширину ячеек
        ws.column_dimensions["A"].width = 11
        ws.column_dimensions["B"].width = 7
        ws.column_dimensions["C"].width = 10
        ws.column_dimensions["D"].width = 10
        formating_cells_col = ['E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
                               'O', 'P', 'Q', 'R', 'S', 'T', 'U']
        for column_of_cells in formating_cells_col:
            ws.column_dimensions[column_of_cells].width = 6
        ws.column_dimensions["V"].width = 10

        
        ws.row_dimensions[8].height = 32.531
        ws.row_dimensions[9].height = 32.531
        
        formating_cells_row = list(range(10, 10 + count_of_meathures))
        for form_row in formating_cells_row:
            ws.row_dimensions[form_row].height = 16
        ##################
        ###
        
        ############
#         ВРЕМЕННО, ЗАТЕМ ВКЛЮЧИТЬ
        name_of_picture_file = path_to_file[:-5]+'.files/Graphs/'+str(well).replace('.0', '')+'.png'
        
        if os.path.exists(name_of_picture_file) == False:
            list_of_plottiong_columns = ['Дата', '0.0', '0.5', '1.0', '1.5', '2.0', '2.5', '3.0', '4.0', '5.0', '6.0', 
                                     '7.0', '8.0', '9.0', '10.0', '12.0', '14.0', '15.0']
            plot_graph(df_of_well, name_of_picture_file, list_of_plottiong_columns)

        scale_image(input_image_path=name_of_picture_file,
                    output_image_path=name_of_picture_file,
                    width=width_for_15m_well,)
        img = Image(name_of_picture_file)
        ws.add_image(img, 'A24')
        #################
        name_of_location_pic_file = path_to_file[:-5]+'.files/Locate/'+str(well).replace('.0', '')+'.png'


        if os.path.exists(name_of_location_pic_file):
            scale_image(input_image_path=name_of_location_pic_file,
                        output_image_path=name_of_location_pic_file,
                        width=width_for_15m_well)
            img = Image(name_of_location_pic_file)
            ws.add_image(img, 'A45')            
        else:
            print('Для скважины', well, 'отсутствует картинка местоположения')

    else:
        # если скважина 10 М
        cell = WriteOnlyCell(ws, value="Результаты измерений температуры:")
        cell.font = Font(name='Arial', size=12, bold = 'True')
        cell.alignment = Alignment(horizontal = 'center')

        ws.append([cell])
        ws.merged_cells.ranges.append("A7:R7")      
    
        ws.append(['Дата замера', '№ Термокосы', 'Высота оголовка, м', 'Температура воздуха, °С', 'Температура в скважине на глубине от поверхности земли, м, °С',
                   '', '', '', '', '', '', '', '', '', '', '', '', '', 'Температура средняя, °С'])
        ws.append(['', '', '', '', '0.0', '0.5', '1.0', '1.5', '2.0', '2.5', '3.0', '4.0', '5.0', '6.0', '7.0', '8.0', '9.0', '10.0'])
        
        for index, row in df_of_well.iterrows():
            if row['Дата'] !='-':
                date_value = row['Дата'].strftime("%d.%m.%Y")
            else:
                date_value = row['Дата']
            ws.append([date_value, str(row['№ Термокосы']).replace('.0', ''), row['Высота ТС от уровня земли, м'], row['Температура воздуха, °С'],
                       row['0.0'], row['0.5'], row['1.0'], row['1.5'], row['2.0'], row['2.5'], row['3.0'], row['4.0'], row['5.0'], row['6.0'], row['7.0'], row['8.0'], row['9.0'], row['10.0'], get_temperature_mean()])

        ws.merged_cells.ranges.append("A8:A9")
        ws.merged_cells.ranges.append("B8:B9")
        ws.merged_cells.ranges.append("C8:C9")
        ws.merged_cells.ranges.append("D8:D9")
        ws.merged_cells.ranges.append("S8:S9")        
        ws.merged_cells.ranges.append("E8:R8")
        ######################
        #формируем таблицу
        thin = Side(border_style="thin", color="000000")
        double = Side(border_style="double", color="000000")
        medium = Side(border_style="medium", color="000000")
        
        formating_cells = ['A8']
        for formating_cell_position in formating_cells:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(top=medium, left=medium, right=thin, bottom=thin)
            standart_formating_cell(formating_cell)
            
        formating_cells = ['B8','C8', 'D8', 'E8','F8', 'G8', 'H8','I8', 'J8', 'K8','L8', 'M8',
                           'N8','O8', 'P8', 'R8','Q8',]
        for formating_cell_position in formating_cells:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(top=medium, left=thin, right=thin, bottom=thin)
            standart_formating_cell(formating_cell)
            
        formating_cells = ['S8']
        for formating_cell_position in formating_cells:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(top=medium, left=thin, right=medium, bottom=thin)
            standart_formating_cell(formating_cell)
           
        formating_cells = ['A9']
        for formating_cell_position in formating_cells:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(top=thin, left=medium, right=thin, bottom=thin)

            formating_cells = ['B9','C9', 'D9', 'E9','F9', 'G9', 'H9','I9', 'J9', 'K9','L9', 'M9',
                           'N9','O9', 'P9', 'Q9','R9',]
        for formating_cell_position in formating_cells:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
            standart_formating_cell(formating_cell)
        
        formating_cells = ['S9']
        for formating_cell_position in formating_cells:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(top=thin, left=thin, right=medium, bottom=thin)
            standart_formating_cell(formating_cell)
        
        count_of_meathures = len(df_of_well)
        formating_cells_col = ['A']
        formating_cells_row = list(range(10, 10+count_of_meathures))
        formating_cells = get_formatting_cells(formating_cells_col, formating_cells_row)
        for ind, formating_cell_position in enumerate (formating_cells):
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(left=medium, right=thin)
            standart_formating_cell(formating_cell)
            if str(formating_cells_row[-1]) in formating_cell_position:
                formating_cell = ws[formating_cell_position]
                formating_cell.border = Border(left=medium, right=thin, bottom=medium)
                standart_formating_cell(formating_cell)
        
        count_of_meathures = len(df_of_well)
        formating_cells_col = ['B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
                               'O', 'P', 'Q', 'R']
        formating_cells_row = list(range(10, 10+count_of_meathures))
        formating_cells = get_formatting_cells(formating_cells_col, formating_cells_row)
        for ind, formating_cell_position in enumerate (formating_cells):
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(left=thin, right=thin)
            standart_formating_cell(formating_cell)
            if str(formating_cells_row[-1]) in formating_cell_position:
                formating_cell = ws[formating_cell_position]
                formating_cell.border = Border(left=thin, right=thin, bottom=medium)
                standart_formating_cell(formating_cell)
                
                
        formating_cells_col = ['S']
        formating_cells_row = list(range(10, 10+count_of_meathures))
        formating_cells = get_formatting_cells(formating_cells_col, formating_cells_row)
        for ind, formating_cell_position in enumerate (formating_cells):
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(left=thin, right=medium)
            standart_formating_cell(formating_cell)
            if str(formating_cells_row[-1]) in formating_cell_position:
                formating_cell = ws[formating_cell_position]
                formating_cell.border = Border(left=thin, right=medium, bottom=medium)
                standart_formating_cell(formating_cell)
        
        # Настраиваем ширину ячеек
        ws.column_dimensions["A"].width = 11
        ws.column_dimensions["B"].width = 7
        ws.column_dimensions["C"].width = 10
        ws.column_dimensions["D"].width = 10
        formating_cells_col = ['E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
                               'O', 'P', 'Q', 'R']
        for column_of_cells in formating_cells_col:
            ws.column_dimensions[column_of_cells].width = 6
        ws.column_dimensions["S"].width = 10

        
        ws.row_dimensions[8].height = 32.531
        ws.row_dimensions[9].height = 32.531
        
        formating_cells_row = list(range(10, 10 + count_of_meathures))
        for form_row in formating_cells_row:
            ws.row_dimensions[form_row].height = 16
        ##################
        ###
        
        ############
#         ВРЕМЕННО, ЗАТЕМ ВКЛЮЧИТЬ
        name_of_picture_file = path_to_file[:-5]+'.files/Graphs/'+str(well)+'.png'
        
        if os.path.exists(name_of_picture_file) == False:
            list_of_plottiong_columns = ['Дата', '0.0', '0.5', '1.0', '1.5', '2.0', '2.5', '3.0', '4.0', '5.0', '6.0', 
                                     '7.0', '8.0', '9.0', '10.0']
            plot_graph(df_of_well, name_of_picture_file, list_of_plottiong_columns)
        scale_image(input_image_path=name_of_picture_file,
                    output_image_path=name_of_picture_file,
                    width=width_for_10m_well,)
        img = Image(name_of_picture_file)
        ws.add_image(img, 'A24')
        #################
        name_of_location_pic_file = path_to_file[:-5]+'.files/Locate/'+str(well).replace('Т', '')+'.png'
        if os.path.exists(name_of_location_pic_file):
            scale_image(input_image_path=name_of_location_pic_file,
                        output_image_path=name_of_location_pic_file,
                        width=width_for_15m_well)  
            img = Image(name_of_location_pic_file)
            ws.add_image(img, 'A45')
        else:
            print('Для скважины', well, 'отсутствует картинка местоположения')
        
                  
wb.save(name_of_passport_file)

print('Vse')

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
15 м скважина
Vse


## 5. Обрабатываем напорник

In [8]:
path_to_file = 'Том 10/Том X_Приложение А,Б_НН (ПК217.1 - 357.3).xlsx' # относительный путь до приложения
list_of_table_names = ['НН ПК217.1-ПК357.3 Весна 2018',
                       'НН ПК217.1-ПК357.3 Весна 2019',
                       'НН ПК217.1-ПК357.3 Осень 2019',
                       'НН ПК217.1-ПК357.3 Осень 2020', 
                       'НН ПК217.1-ПК357.3 весна 2021'] #обрабатываемые листы в порядке заполнения пасспорта

def rename_df(df):
    # переименовываем и удаляем ненужные колонки
    df.rename(columns = {'Unnamed: 1' : 'Позиция по ГП', 'Unnamed: 2':'Скважина', 'Дата.1' : 'ДАТА', 'Unnamed: 13' : 'Высота ТС от уровня земли, м', 
                         'Unnamed: 9' : '№ Термокосы', 'Unnamed: 20' : 'Температура воздуха, °С',
                     0:str('0.0'), 0.5:str('0.5'), 1:str('1.0'), 1.5:str('1.5'), 2:str('2.0'), 2.5:str('2.5'), 3 : str('3.0'), 
                     4:str('4.0'), 5:str('5.0'), 6:str('6.0'), 7:str('7.0'), 8:str('8.0'), 9:str('9.0'), 
                     10:str('10.0'), 12:str('12.0'), 14:str('14.0'), 15:str('15.0')}, inplace = True)
    
    actual_columns = ['Позиция по ГП', 'Скважина', 'ДАТА', 'Высота ТС от уровня земли, м', '0.0', '№ Термокосы', 'Температура воздуха, °С', '0.5', '1.0', '1.5', '2.0', '2.5', '3.0', 
                     '4.0', '5.0', '6.0', '7.0', '8.0', '9.0', 
                     '10.0', '12.0', '14.0', '15.0']
    
    current_col = list(df.columns)
    for temp in current_col:
        dropped_col = []
        if temp not in actual_columns:
            dropped_col.append(temp)
        df.drop(columns = dropped_col, inplace = True)
    df.rename(columns = {'ДАТА' : 'Дата'}, inplace = True)
    


### Том 10

In [60]:
path_to_file = 'Том 10/Том X_Приложение А,Б_НН (ПК217.1 - 357.3).xlsx' # относительный путь до приложения
list_of_table_names = ['НН ПК217.1-ПК357.3 Весна 2018',
                       'НН ПК217.1-ПК357.3 Весна 2019',
                       'НН ПК217.1-ПК357.3 Осень 2019',
                       'НН ПК217.1-ПК357.3 Осень 2020', 
                       'НН ПК217.1-ПК357.3 весна 2021'] #обрабатываемые листы в порядке заполнения пасспорта

for i, sheet in enumerate (list_of_table_names):
    if i==0:
        df = pd.read_excel(path_to_file, sheet_name = sheet, skiprows = 12, date_parser = lambda x: pd.datetime.strptime(x, "%d.%m.%Y")) # вроде вставка дат работает как надо
        rename_df(df)
    else:
        df_temp = pd.read_excel(path_to_file, sheet_name = sheet, skiprows = 12, date_parser = lambda x: pd.datetime.strptime(x, "%d.%m.%Y")) # вроде вставка дат работает как надо
        rename_df(df_temp)
        df = pd.concat([df, df_temp])
    df = df[df['Дата'].notna()]
######


colour_list = ['00800080', '00008080', '00C0C0C0', '00808080', '009999FF', '00CCFFFF', '00660066', '00FF8080', '000066CC', '00CCCCFF', '00000080', '00FF00FF', '00FFFF00']

name_of_passport_file = path_to_file[:-5]+'_passports'+ path_to_file[-5:]
wb = openpyxl.Workbook()
list_of_unique_wells = list(df['Скважина'].dropna().unique())
# list_of_unique_wells = list_of_unique_wells[180:190] ##### временно
for well in list_of_unique_wells:
    df_of_well = df[df['Скважина'] == well] # пока обойдемся без сортировки

    ### Паспорт термометрической скважины:
    ws = wb.create_sheet(title = str(well).replace('.0', ''))
    cell = WriteOnlyCell(ws, value="Паспорт термометрической скважины:")
    cell.font = Font(name='Arial', size=14, bold = 'True')
    cell.alignment = Alignment(horizontal = 'right')

    cell_well = WriteOnlyCell(ws, value=well)
    cell_well.font = Font(name='Arial', size=12, bold = 'True')
    cell_well.alignment = Alignment(horizontal = 'center')
    cell_well.border = Border(bottom=Side(border_style='thin', color='FF000000'),)
    cell_Border = WriteOnlyCell(ws, value='')
    cell_Border.border = Border(bottom=Side(border_style='thin', color='FF000000'),)

    ws.append(['']) #первый отступ
    ws.append([cell, '', '', '', '', '', '', '', '', '', '', '', '', cell_well, cell_Border])
    ws.merged_cells.ranges.append("A2:M2")
    ws.merged_cells.ranges.append("N2:O2")

    ### Местоположение термоскважины
    ws.append([])

    cell = WriteOnlyCell(ws, value="Местоположение термоскважины:")
    cell.font = Font(name='Arial', size=12, bold = 'True')
    cell.alignment = Alignment(horizontal = 'right')

    name_of_area = 'Площадка ПСП'
    cell_area = WriteOnlyCell(ws, value=name_of_area)
    cell_area.font = Font(name='Arial', size=12)
    cell_area.alignment = Alignment(horizontal = 'left')

    ws.append([cell, '', '', '', '', '', '', cell_area])
    ws.merged_cells.ranges.append("A4:G4")
    ws.merged_cells.ranges.append("H4:R4")

    cell = WriteOnlyCell(ws, value=df_of_well['Позиция по ГП'].iloc[-1])
    cell.font = Font(name='Arial', size=12)
    cell.alignment = Alignment(horizontal = 'left')
    ws.append(['', '', '', '', '', '', '', cell])
    ws.merged_cells.ranges.append("H5:R5")

    ### формируем таблицу
    ws.append([])


    # если скважина 10 М
    cell = WriteOnlyCell(ws, value="Результаты измерений температуры:")
    cell.font = Font(name='Arial', size=12, bold = 'True')
    cell.alignment = Alignment(horizontal = 'center')

    ws.append([cell])
    ws.merged_cells.ranges.append("A7:R7")      

    ws.append(['Дата замера', '№ Термокосы', 'Высота оголовка, м', 'Температура воздуха, °С', 'Температура в скважине на глубине от поверхности земли, м, °С',
               '', '', '', '', '', '', '', '', '', '', '', '', '', 'Температура средняя, °С'])
    ws.append(['', '', '', '', '0.0', '0.5', '1.0', '1.5', '2.0', '2.5', '3.0', '4.0', '5.0', '6.0', '7.0', '8.0', '9.0', '10.0'])

    for index, row in df_of_well.iterrows():
        if row['Дата'] !='-':
            date_value = row['Дата'].strftime("%d.%m.%Y")
        else:
            date_value = row['Дата']
        ws.append([date_value, str(row['№ Термокосы']).replace('.0', ''), row['Высота ТС от уровня земли, м'], row['Температура воздуха, °С'],
                   row['0.0'], row['0.5'], row['1.0'], row['1.5'], row['2.0'], row['2.5'], row['3.0'], row['4.0'], row['5.0'], row['6.0'], row['7.0'], row['8.0'], row['9.0'], row['10.0'], get_temperature_mean()])

    ws.merged_cells.ranges.append("A8:A9")
    ws.merged_cells.ranges.append("B8:B9")
    ws.merged_cells.ranges.append("C8:C9")
    ws.merged_cells.ranges.append("D8:D9")
    ws.merged_cells.ranges.append("S8:S9")        
    ws.merged_cells.ranges.append("E8:R8")
    ######################
    #формируем таблицу
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    medium = Side(border_style="medium", color="000000")

    formating_cells = ['A8']
    for formating_cell_position in formating_cells:
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(top=medium, left=medium, right=thin, bottom=thin)
        standart_formating_cell(formating_cell)

    formating_cells = ['B8','C8', 'D8', 'E8','F8', 'G8', 'H8','I8', 'J8', 'K8','L8', 'M8',
                       'N8','O8', 'P8', 'R8','Q8',]
    for formating_cell_position in formating_cells:
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(top=medium, left=thin, right=thin, bottom=thin)
        standart_formating_cell(formating_cell)

    formating_cells = ['S8']
    for formating_cell_position in formating_cells:
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(top=medium, left=thin, right=medium, bottom=thin)
        standart_formating_cell(formating_cell)

    formating_cells = ['A9']
    for formating_cell_position in formating_cells:
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(top=thin, left=medium, right=thin, bottom=thin)

        formating_cells = ['B9','C9', 'D9', 'E9','F9', 'G9', 'H9','I9', 'J9', 'K9','L9', 'M9',
                       'N9','O9', 'P9', 'Q9','R9',]
    for formating_cell_position in formating_cells:
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
        standart_formating_cell(formating_cell)

    formating_cells = ['S9']
    for formating_cell_position in formating_cells:
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(top=thin, left=thin, right=medium, bottom=thin)
        standart_formating_cell(formating_cell)

    count_of_meathures = len(df_of_well)
    formating_cells_col = ['A']
    formating_cells_row = list(range(10, 10+count_of_meathures))
    formating_cells = get_formatting_cells(formating_cells_col, formating_cells_row)
    for ind, formating_cell_position in enumerate (formating_cells):
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(left=medium, right=thin)
        standart_formating_cell(formating_cell)
        if str(formating_cells_row[-1]) in formating_cell_position:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(left=medium, right=thin, bottom=medium)
            standart_formating_cell(formating_cell)

    count_of_meathures = len(df_of_well)
    formating_cells_col = ['B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
                           'O', 'P', 'Q', 'R']
    formating_cells_row = list(range(10, 10+count_of_meathures))
    formating_cells = get_formatting_cells(formating_cells_col, formating_cells_row)
    for ind, formating_cell_position in enumerate (formating_cells):
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(left=thin, right=thin)
        standart_formating_cell(formating_cell)
        if str(formating_cells_row[-1]) in formating_cell_position:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(left=thin, right=thin, bottom=medium)
            standart_formating_cell(formating_cell)


    formating_cells_col = ['S']
    formating_cells_row = list(range(10, 10+count_of_meathures))
    formating_cells = get_formatting_cells(formating_cells_col, formating_cells_row)
    for ind, formating_cell_position in enumerate (formating_cells):
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(left=thin, right=medium)
        standart_formating_cell(formating_cell)
        if str(formating_cells_row[-1]) in formating_cell_position:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(left=thin, right=medium, bottom=medium)
            standart_formating_cell(formating_cell)

    # Настраиваем ширину ячеек
    ws.column_dimensions["A"].width = 11
    ws.column_dimensions["B"].width = 7
    ws.column_dimensions["C"].width = 10
    ws.column_dimensions["D"].width = 10
    formating_cells_col = ['E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
                           'O', 'P', 'Q', 'R']
    for column_of_cells in formating_cells_col:
        ws.column_dimensions[column_of_cells].width = 6
    ws.column_dimensions["S"].width = 10


    ws.row_dimensions[8].height = 32.531
    ws.row_dimensions[9].height = 32.531

    formating_cells_row = list(range(10, 10 + count_of_meathures))
    for form_row in formating_cells_row:
        ws.row_dimensions[form_row].height = 16
    ##################
    ###

    ############
#         ВРЕМЕННО, ЗАТЕМ ВКЛЮЧИТЬ
    name_of_picture_file = path_to_file[:-5]+'.files/Graphs/'+str(well)+'.png'

    if os.path.exists(name_of_picture_file) == False:
        list_of_plottiong_columns = ['Дата', '0.0', '0.5', '1.0', '1.5', '2.0', '2.5', '3.0', '4.0', '5.0', '6.0', 
                                 '7.0', '8.0', '9.0', '10.0']
        os.makedirs(path_to_file[:-5]+'.files/Graphs/' ,exist_ok=True)
        plot_graph(df_of_well, name_of_picture_file, list_of_plottiong_columns)
    scale_image(input_image_path=name_of_picture_file,
                output_image_path=name_of_picture_file,
                width=width_for_10m_well,)
    img = Image(name_of_picture_file)
    ws.add_image(img, 'A24')
    #################
    name_of_location_pic_file = path_to_file[:-5]+'.files/Locate/'+str(well).replace('Т', '')+'.png'
    os.makedirs(path_to_file[:-5]+'.files/Locate/' ,exist_ok=True)
    if os.path.exists(name_of_location_pic_file):
        scale_image(input_image_path=name_of_location_pic_file,
                    output_image_path=name_of_location_pic_file,
                    width=width_for_10m_well)  
        img = Image(name_of_location_pic_file)
        ws.add_image(img, 'A45')
    else:
        print('Для скважины', well, 'отсутствует картинка местоположения')


wb.save(name_of_passport_file)

print('Vse')

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


Для скважины 217.1 отсутствует картинка местоположения
Для скважины 217.2 отсутствует картинка местоположения
Для скважины 218.1 отсутствует картинка местоположения
Для скважины 218.2 отсутствует картинка местоположения
Для скважины 219.1 отсутствует картинка местоположения
Для скважины 219.2 отсутствует картинка местоположения
Для скважины 221.1 отсутствует картинка местоположения
Для скважины 221.2 отсутствует картинка местоположения
Для скважины 221.3 отсутствует картинка местоположения
Для скважины 222.2 отсутствует картинка местоположения
Для скважины 224.1 отсутствует картинка местоположения
Для скважины 224.2 отсутствует картинка местоположения
Для скважины 224.3 отсутствует картинка местоположения
Для скважины 225.1 отсутствует картинка местоположения
Для скважины 225.2 отсутствует картинка местоположения
Для скважины 226.2 отсутствует картинка местоположения
Для скважины 227.2 отсутствует картинка местоположения
Для скважины 227.3 отсутствует картинка местоположения
Для скважи

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


save_pic 273.1
Для скважины 273.1 отсутствует картинка местоположения
Для скважины 273.2 отсутствует картинка местоположения
Для скважины 274.1 отсутствует картинка местоположения
Для скважины 274.2 отсутствует картинка местоположения
Для скважины 276.2 отсутствует картинка местоположения
Для скважины 276.3 отсутствует картинка местоположения
Для скважины 277.1 отсутствует картинка местоположения
Для скважины 278.1 отсутствует картинка местоположения
Для скважины 278.2 отсутствует картинка местоположения
Для скважины 278.3 отсутствует картинка местоположения
Для скважины 279.1 отсутствует картинка местоположения
Для скважины 279.2 отсутствует картинка местоположения
Для скважины 280.1 отсутствует картинка местоположения
Для скважины 281.1 отсутствует картинка местоположения
Для скважины 281.2 отсутствует картинка местоположения
Для скважины 282.1 отсутствует картинка местоположения
Для скважины 283.2 отсутствует картинка местоположения
Для скважины 284.1 отсутствует картинка местополож

### Том 11

In [27]:
def get_and_cheked_sheets(path_to_file):
    list_of_table_names = pd.ExcelFile(path_to_file).sheet_names #обрабатываемые листы в порядке заполнения пасспорта (должны остаться только листы со сводными)
    temp = list()
    for i, name in enumerate (list_of_table_names):        
        if ('Весна' in name) or ('Осень' in name):
            temp.append(name)
    return(sorted(temp))

path_to_file = 'Том 11/Том XI_Приложение А,Б_НН (ПК 511.2-728.1).xlsx' # относительный путь до приложения
list_of_table_names = ['НН ПК 511.2-728.1 Осень 2018', 'НН ПК 511.2-728.1 Весна 2019', 'НН ПК 511.2-728.1 Весна 2020',  'НН ПК 511.2-728.1 Осень 2020', 'НН ПК 511.2-728.1 Весна 2021']
width_for_10m_well = 950
for i, sheet in enumerate (list_of_table_names):
    if i==0:
        df = pd.read_excel(path_to_file, sheet_name = sheet, skiprows = 12, date_parser = lambda x: pd.datetime.strptime(x, "%d.%m.%Y")) # вроде вставка дат работает как надо
        rename_df(df)
    else:
        df_temp = pd.read_excel(path_to_file, sheet_name = sheet, skiprows = 12, date_parser = lambda x: pd.datetime.strptime(x, "%d.%m.%Y")) # вроде вставка дат работает как надо
        rename_df(df_temp)
        df = pd.concat([df, df_temp])
    df = df[df['Дата'].notna()]

######
df['Скважина'] == df['Скважина'].astype(str)
name_of_passport_file = path_to_file[:-5]+'_passports'+ path_to_file[-5:]
wb = openpyxl.Workbook()
list_of_unique_wells = list(df['Скважина'].dropna().unique())
# list_of_unique_wells = list_of_unique_wells[180:190] ##### временно
for well in list_of_unique_wells:
    df_of_well = df[df['Скважина'] == well] # пока обойдемся без сортировки

    ### Паспорт термометрической скважины:
    ws = wb.create_sheet(title = str(well)) #правка
    cell = WriteOnlyCell(ws, value="Паспорт термометрической скважины:")
    cell.font = Font(name='Arial', size=14, bold = 'True')
    cell.alignment = Alignment(horizontal = 'right')

    cell_well = WriteOnlyCell(ws, value=well)
    cell_well.font = Font(name='Arial', size=12, bold = 'True')
    cell_well.alignment = Alignment(horizontal = 'center')
    cell_well.border = Border(bottom=Side(border_style='thin', color='FF000000'),)
    cell_Border = WriteOnlyCell(ws, value='')
    cell_Border.border = Border(bottom=Side(border_style='thin', color='FF000000'),)

    ws.append(['']) #первый отступ
    ws.append([cell, '', '', '', '', '', '', '', '', '', '', '', '', cell_well, cell_Border])
    ws.merged_cells.ranges.append("A2:M2")
    ws.merged_cells.ranges.append("N2:O2")

    ### Местоположение термоскважины
    ws.append([])

    cell = WriteOnlyCell(ws, value="Местоположение термоскважины:")
    cell.font = Font(name='Arial', size=12, bold = 'True')
    cell.alignment = Alignment(horizontal = 'right')

    name_of_area = 'Площадка ПСП'
    cell_area = WriteOnlyCell(ws, value=name_of_area)
    cell_area.font = Font(name='Arial', size=12)
    cell_area.alignment = Alignment(horizontal = 'left')

    ws.append([cell, '', '', '', '', '', '', cell_area])
    ws.merged_cells.ranges.append("A4:G4")
    ws.merged_cells.ranges.append("H4:R4")

    cell = WriteOnlyCell(ws, value=df_of_well['Позиция по ГП'].iloc[-1])
    cell.font = Font(name='Arial', size=12)
    cell.alignment = Alignment(horizontal = 'left')
    ws.append(['', '', '', '', '', '', '', cell])
    ws.merged_cells.ranges.append("H5:R5")

    ### формируем таблицу
    ws.append([])


    # если скважина 10 М
    cell = WriteOnlyCell(ws, value="Результаты измерений температуры:")
    cell.font = Font(name='Arial', size=12, bold = 'True')
    cell.alignment = Alignment(horizontal = 'center')

    ws.append([cell])
    ws.merged_cells.ranges.append("A7:R7")      

    ws.append(['Дата замера', '№ Термокосы', 'Высота оголовка, м', 'Температура воздуха, °С', 'Температура в скважине на глубине от поверхности земли, м, °С',
               '', '', '', '', '', '', '', '', '', '', '', '', '', 'Температура средняя, °С'])
    ws.append(['', '', '', '', '0.0', '0.5', '1.0', '1.5', '2.0', '2.5', '3.0', '4.0', '5.0', '6.0', '7.0', '8.0', '9.0', '10.0'])

    for index, row in df_of_well.iterrows():
        if row['Дата'] !='-':
            date_value = row['Дата'].strftime("%d.%m.%Y")
        else:
            date_value = row['Дата']
        ws.append([date_value, str(row['№ Термокосы']).replace('.0', ''), row['Высота ТС от уровня земли, м'], row['Температура воздуха, °С'],
                   row['0.0'], row['0.5'], row['1.0'], row['1.5'], row['2.0'], row['2.5'], row['3.0'], row['4.0'], row['5.0'], row['6.0'], row['7.0'], row['8.0'], row['9.0'], row['10.0'], get_temperature_mean()])

    ws.merged_cells.ranges.append("A8:A9")
    ws.merged_cells.ranges.append("B8:B9")
    ws.merged_cells.ranges.append("C8:C9")
    ws.merged_cells.ranges.append("D8:D9")
    ws.merged_cells.ranges.append("S8:S9")        
    ws.merged_cells.ranges.append("E8:R8")
    ######################
    #формируем таблицу
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    medium = Side(border_style="medium", color="000000")

    formating_cells = ['A8']
    for formating_cell_position in formating_cells:
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(top=medium, left=medium, right=thin, bottom=thin)
        standart_formating_cell(formating_cell)

    formating_cells = ['B8','C8', 'D8', 'E8','F8', 'G8', 'H8','I8', 'J8', 'K8','L8', 'M8',
                       'N8','O8', 'P8', 'R8','Q8',]
    for formating_cell_position in formating_cells:
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(top=medium, left=thin, right=thin, bottom=thin)
        standart_formating_cell(formating_cell)

    formating_cells = ['S8']
    for formating_cell_position in formating_cells:
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(top=medium, left=thin, right=medium, bottom=thin)
        standart_formating_cell(formating_cell)

    formating_cells = ['A9']
    for formating_cell_position in formating_cells:
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(top=thin, left=medium, right=thin, bottom=thin)

        formating_cells = ['B9','C9', 'D9', 'E9','F9', 'G9', 'H9','I9', 'J9', 'K9','L9', 'M9',
                       'N9','O9', 'P9', 'Q9','R9',]
    for formating_cell_position in formating_cells:
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
        standart_formating_cell(formating_cell)

    formating_cells = ['S9']
    for formating_cell_position in formating_cells:
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(top=thin, left=thin, right=medium, bottom=thin)
        standart_formating_cell(formating_cell)

    count_of_meathures = len(df_of_well)
    formating_cells_col = ['A']
    formating_cells_row = list(range(10, 10+count_of_meathures))
    formating_cells = get_formatting_cells(formating_cells_col, formating_cells_row)
    for ind, formating_cell_position in enumerate (formating_cells):
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(left=medium, right=thin)
        standart_formating_cell(formating_cell)
        if str(formating_cells_row[-1]) in formating_cell_position:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(left=medium, right=thin, bottom=medium)
            standart_formating_cell(formating_cell)

    count_of_meathures = len(df_of_well)
    formating_cells_col = ['B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
                           'O', 'P', 'Q', 'R']
    formating_cells_row = list(range(10, 10+count_of_meathures))
    formating_cells = get_formatting_cells(formating_cells_col, formating_cells_row)
    for ind, formating_cell_position in enumerate (formating_cells):
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(left=thin, right=thin)
        standart_formating_cell(formating_cell)
        if str(formating_cells_row[-1]) in formating_cell_position:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(left=thin, right=thin, bottom=medium)
            standart_formating_cell(formating_cell)


    formating_cells_col = ['S']
    formating_cells_row = list(range(10, 10+count_of_meathures))
    formating_cells = get_formatting_cells(formating_cells_col, formating_cells_row)
    for ind, formating_cell_position in enumerate (formating_cells):
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(left=thin, right=medium)
        standart_formating_cell(formating_cell)
        if str(formating_cells_row[-1]) in formating_cell_position:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(left=thin, right=medium, bottom=medium)
            standart_formating_cell(formating_cell)

    # Настраиваем ширину ячеек
    ws.column_dimensions["A"].width = 11
    ws.column_dimensions["B"].width = 7
    ws.column_dimensions["C"].width = 10
    ws.column_dimensions["D"].width = 10
    formating_cells_col = ['E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
                           'O', 'P', 'Q', 'R']
    for column_of_cells in formating_cells_col:
        ws.column_dimensions[column_of_cells].width = 6
    ws.column_dimensions["S"].width = 10


    ws.row_dimensions[8].height = 32.531
    ws.row_dimensions[9].height = 32.531

    formating_cells_row = list(range(10, 10 + count_of_meathures))
    for form_row in formating_cells_row:
        ws.row_dimensions[form_row].height = 16
    ##################
    ###

    ############
#         ВРЕМЕННО, ЗАТЕМ ВКЛЮЧИТЬ
    name_of_picture_file = path_to_file[:-5]+'.files/Graphs/'+str(well)+'.png'

    if os.path.exists(name_of_picture_file) == False:
        list_of_plottiong_columns = ['Дата', '0.0', '0.5', '1.0', '1.5', '2.0', '2.5', '3.0', '4.0', '5.0', '6.0', 
                                 '7.0', '8.0', '9.0', '10.0']
        os.makedirs(path_to_file[:-5]+'.files/Graphs/' ,exist_ok=True)
        plot_graph(df_of_well, name_of_picture_file, list_of_plottiong_columns)
    scale_image(input_image_path=name_of_picture_file,
                output_image_path=name_of_picture_file,
                width=width_for_10m_well,)
    img = Image(name_of_picture_file)
    ws.add_image(img, 'A24')
    #################
    name_of_location_pic_file = path_to_file[:-5]+'.files/Locate/'+str(well)+'.png'
    os.makedirs(path_to_file[:-5]+'.files/Locate/' ,exist_ok=True)
    if os.path.exists(name_of_location_pic_file):
        scale_image(input_image_path=name_of_location_pic_file,
                    output_image_path=name_of_location_pic_file,
                    width=width_for_10m_well)  
        img = Image(name_of_location_pic_file)
        ws.add_image(img, 'A45')
    else:
        print('Для скважины', well, 'отсутствует картинка местоположения')


wb.save(name_of_passport_file)

print('Vse')

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


save_pic 538.1
Для скважины 538.1 отсутствует картинка местоположения
save_pic 539.1
Для скважины 539.1 отсутствует картинка местоположения
save_pic 539.2
Для скважины 539.2 отсутствует картинка местоположения
save_pic 541.1
Для скважины 541.1 отсутствует картинка местоположения
save_pic 541.2
Для скважины 541.2 отсутствует картинка местоположения
save_pic 542.1
Для скважины 542.1 отсутствует картинка местоположения
save_pic 543.1
Для скважины 543.1 отсутствует картинка местоположения
save_pic 544.1
Для скважины 544.1 отсутствует картинка местоположения
save_pic 544.2
Для скважины 544.2 отсутствует картинка местоположения
save_pic 545.1
Для скважины 545.1 отсутствует картинка местоположения
save_pic 545.2
Для скважины 545.2 отсутствует картинка местоположения
save_pic 546.1
Для скважины 546.1 отсутствует картинка местоположения
save_pic 546.2
Для скважины 546.2 отсутствует картинка местоположения
save_pic 547.1
Для скважины 547.1 отсутствует картинка местоположения
save_pic 548.2
Для с

### Том 12

In [26]:
path_to_file = 'Том 12/Том XII_Приложение А,Б_НН (ПК728.2-955.3).xlsx' # относительный путь до приложения
list_of_table_names = ['НН Осень 2018 ПК728.2-955.3', 'НН Весна 2018 ПК728.2-955.3', 'НН Осень 2019 ПК728.2-955.3', 'НН Весна 2019 ПК728.2-955.3',  
                       'НН Весна 2020 ПК728.2-955.3', 'НН Осень 2020 ПК728.2-955.3', 'НН Весна 2021 ПК728.2-955.3']
width_for_10m_well = 950
for i, sheet in enumerate (list_of_table_names):
    if i==0:
        df = pd.read_excel(path_to_file, sheet_name = sheet, skiprows = 12, date_parser = lambda x: pd.datetime.strptime(x, "%d.%m.%Y")) # вроде вставка дат работает как надо
        rename_df(df)
    else:
        df_temp = pd.read_excel(path_to_file, sheet_name = sheet, skiprows = 12, date_parser = lambda x: pd.datetime.strptime(x, "%d.%m.%Y")) # вроде вставка дат работает как надо
        rename_df(df_temp)
        df = pd.concat([df, df_temp])
    df = df[df['Дата'].notna()]

######
df['Скважина'] == df['Скважина'].astype(str)
name_of_passport_file = path_to_file[:-5]+'_passports'+ path_to_file[-5:]
wb = openpyxl.Workbook()
list_of_unique_wells = list(df['Скважина'].dropna().unique())
# list_of_unique_wells = list_of_unique_wells[180:190] ##### временно
for well in list_of_unique_wells:
    df_of_well = df[df['Скважина'] == well] # пока обойдемся без сортировки

    ### Паспорт термометрической скважины:
    ws = wb.create_sheet(title = str(well).replace('/', '')) #правка
    cell = WriteOnlyCell(ws, value="Паспорт термометрической скважины:")
    cell.font = Font(name='Arial', size=14, bold = 'True')
    cell.alignment = Alignment(horizontal = 'right')

    cell_well = WriteOnlyCell(ws, value=well)
    cell_well.font = Font(name='Arial', size=12, bold = 'True')
    cell_well.alignment = Alignment(horizontal = 'center')
    cell_well.border = Border(bottom=Side(border_style='thin', color='FF000000'),)
    cell_Border = WriteOnlyCell(ws, value='')
    cell_Border.border = Border(bottom=Side(border_style='thin', color='FF000000'),)

    ws.append(['']) #первый отступ
    ws.append([cell, '', '', '', '', '', '', '', '', '', '', '', '', cell_well, cell_Border])
    ws.merged_cells.ranges.append("A2:M2")
    ws.merged_cells.ranges.append("N2:O2")

    ### Местоположение термоскважины
    ws.append([])

    cell = WriteOnlyCell(ws, value="Местоположение термоскважины:")
    cell.font = Font(name='Arial', size=12, bold = 'True')
    cell.alignment = Alignment(horizontal = 'right')

    name_of_area = 'Площадка ПСП'
    cell_area = WriteOnlyCell(ws, value=name_of_area)
    cell_area.font = Font(name='Arial', size=12)
    cell_area.alignment = Alignment(horizontal = 'left')

    ws.append([cell, '', '', '', '', '', '', cell_area])
    ws.merged_cells.ranges.append("A4:G4")
    ws.merged_cells.ranges.append("H4:R4")

    cell = WriteOnlyCell(ws, value=df_of_well['Позиция по ГП'].iloc[-1])
    cell.font = Font(name='Arial', size=12)
    cell.alignment = Alignment(horizontal = 'left')
    ws.append(['', '', '', '', '', '', '', cell])
    ws.merged_cells.ranges.append("H5:R5")

    ### формируем таблицу
    ws.append([])


    # если скважина 10 М
    cell = WriteOnlyCell(ws, value="Результаты измерений температуры:")
    cell.font = Font(name='Arial', size=12, bold = 'True')
    cell.alignment = Alignment(horizontal = 'center')

    ws.append([cell])
    ws.merged_cells.ranges.append("A7:R7")      

    ws.append(['Дата замера', '№ Термокосы', 'Высота оголовка, м', 'Температура воздуха, °С', 'Температура в скважине на глубине от поверхности земли, м, °С',
               '', '', '', '', '', '', '', '', '', '', '', '', '', 'Температура средняя, °С'])
    ws.append(['', '', '', '', '0.0', '0.5', '1.0', '1.5', '2.0', '2.5', '3.0', '4.0', '5.0', '6.0', '7.0', '8.0', '9.0', '10.0'])

    for index, row in df_of_well.iterrows():
        if row['Дата'] !='-':
            date_value = row['Дата'].strftime("%d.%m.%Y")
        else:
            date_value = row['Дата']
        ws.append([date_value, str(row['№ Термокосы']).replace('.0', ''), row['Высота ТС от уровня земли, м'], row['Температура воздуха, °С'],
                   row['0.0'], row['0.5'], row['1.0'], row['1.5'], row['2.0'], row['2.5'], row['3.0'], row['4.0'], row['5.0'], row['6.0'], row['7.0'], row['8.0'], row['9.0'], row['10.0'], get_temperature_mean()])

    ws.merged_cells.ranges.append("A8:A9")
    ws.merged_cells.ranges.append("B8:B9")
    ws.merged_cells.ranges.append("C8:C9")
    ws.merged_cells.ranges.append("D8:D9")
    ws.merged_cells.ranges.append("S8:S9")        
    ws.merged_cells.ranges.append("E8:R8")
    ######################
    #формируем таблицу
    thin = Side(border_style="thin", color="000000")
    double = Side(border_style="double", color="000000")
    medium = Side(border_style="medium", color="000000")

    formating_cells = ['A8']
    for formating_cell_position in formating_cells:
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(top=medium, left=medium, right=thin, bottom=thin)
        standart_formating_cell(formating_cell)

    formating_cells = ['B8','C8', 'D8', 'E8','F8', 'G8', 'H8','I8', 'J8', 'K8','L8', 'M8',
                       'N8','O8', 'P8', 'R8','Q8',]
    for formating_cell_position in formating_cells:
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(top=medium, left=thin, right=thin, bottom=thin)
        standart_formating_cell(formating_cell)

    formating_cells = ['S8']
    for formating_cell_position in formating_cells:
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(top=medium, left=thin, right=medium, bottom=thin)
        standart_formating_cell(formating_cell)

    formating_cells = ['A9']
    for formating_cell_position in formating_cells:
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(top=thin, left=medium, right=thin, bottom=thin)

        formating_cells = ['B9','C9', 'D9', 'E9','F9', 'G9', 'H9','I9', 'J9', 'K9','L9', 'M9',
                       'N9','O9', 'P9', 'Q9','R9',]
    for formating_cell_position in formating_cells:
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
        standart_formating_cell(formating_cell)

    formating_cells = ['S9']
    for formating_cell_position in formating_cells:
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(top=thin, left=thin, right=medium, bottom=thin)
        standart_formating_cell(formating_cell)

    count_of_meathures = len(df_of_well)
    formating_cells_col = ['A']
    formating_cells_row = list(range(10, 10+count_of_meathures))
    formating_cells = get_formatting_cells(formating_cells_col, formating_cells_row)
    for ind, formating_cell_position in enumerate (formating_cells):
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(left=medium, right=thin)
        standart_formating_cell(formating_cell)
        if str(formating_cells_row[-1]) in formating_cell_position:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(left=medium, right=thin, bottom=medium)
            standart_formating_cell(formating_cell)

    count_of_meathures = len(df_of_well)
    formating_cells_col = ['B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
                           'O', 'P', 'Q', 'R']
    formating_cells_row = list(range(10, 10+count_of_meathures))
    formating_cells = get_formatting_cells(formating_cells_col, formating_cells_row)
    for ind, formating_cell_position in enumerate (formating_cells):
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(left=thin, right=thin)
        standart_formating_cell(formating_cell)
        if str(formating_cells_row[-1]) in formating_cell_position:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(left=thin, right=thin, bottom=medium)
            standart_formating_cell(formating_cell)


    formating_cells_col = ['S']
    formating_cells_row = list(range(10, 10+count_of_meathures))
    formating_cells = get_formatting_cells(formating_cells_col, formating_cells_row)
    for ind, formating_cell_position in enumerate (formating_cells):
        formating_cell = ws[formating_cell_position]
        formating_cell.border = Border(left=thin, right=medium)
        standart_formating_cell(formating_cell)
        if str(formating_cells_row[-1]) in formating_cell_position:
            formating_cell = ws[formating_cell_position]
            formating_cell.border = Border(left=thin, right=medium, bottom=medium)
            standart_formating_cell(formating_cell)

    # Настраиваем ширину ячеек
    ws.column_dimensions["A"].width = 11
    ws.column_dimensions["B"].width = 7
    ws.column_dimensions["C"].width = 10
    ws.column_dimensions["D"].width = 10
    formating_cells_col = ['E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
                           'O', 'P', 'Q', 'R']
    for column_of_cells in formating_cells_col:
        ws.column_dimensions[column_of_cells].width = 6
    ws.column_dimensions["S"].width = 10


    ws.row_dimensions[8].height = 32.531
    ws.row_dimensions[9].height = 32.531

    formating_cells_row = list(range(10, 10 + count_of_meathures))
    for form_row in formating_cells_row:
        ws.row_dimensions[form_row].height = 16
    ##################
    ###

    ############
#         ВРЕМЕННО, ЗАТЕМ ВКЛЮЧИТЬ
    name_of_picture_file = path_to_file[:-5]+'.files/Graphs/'+str(well)+'.png'

    if os.path.exists(name_of_picture_file) == False:
        list_of_plottiong_columns = ['Дата', '0.0', '0.5', '1.0', '1.5', '2.0', '2.5', '3.0', '4.0', '5.0', '6.0', 
                                 '7.0', '8.0', '9.0', '10.0']
        os.makedirs(path_to_file[:-5]+'.files/Graphs/' ,exist_ok=True)
        plot_graph(df_of_well, name_of_picture_file, list_of_plottiong_columns)
    scale_image(input_image_path=name_of_picture_file,
                output_image_path=name_of_picture_file,
                width=width_for_10m_well,)
    img = Image(name_of_picture_file)
    ws.add_image(img, 'A24')
    #################
    name_of_location_pic_file = path_to_file[:-5]+'.files/Locate/'+str(well)+'.png'
    os.makedirs(path_to_file[:-5]+'.files/Locate/' ,exist_ok=True)
    if os.path.exists(name_of_location_pic_file):
        scale_image(input_image_path=name_of_location_pic_file,
                    output_image_path=name_of_location_pic_file,
                    width=width_for_10m_well)  
        img = Image(name_of_location_pic_file)
        ws.add_image(img, 'A45')
    else:
        print('Для скважины', well, 'отсутствует картинка местоположения')


wb.save(name_of_passport_file)

print('Vse')

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


Для скважины 754.2 отсутствует картинка местоположения
Для скважины 758.1 отсутствует картинка местоположения
Для скважины 758.2 отсутствует картинка местоположения
Для скважины 759.1 отсутствует картинка местоположения
Для скважины 759.2 отсутствует картинка местоположения
Для скважины 759.3 отсутствует картинка местоположения
Для скважины 760.1 отсутствует картинка местоположения
Для скважины 761.1 отсутствует картинка местоположения
Для скважины 761.2 отсутствует картинка местоположения
Для скважины 762.2 отсутствует картинка местоположения
Для скважины 770.1 отсутствует картинка местоположения
Для скважины 770.2 отсутствует картинка местоположения
Для скважины 771.1 отсутствует картинка местоположения
Для скважины 771.2 отсутствует картинка местоположения
Для скважины 772.1 отсутствует картинка местоположения
Для скважины 772.2 отсутствует картинка местоположения
Для скважины 772.3 отсутствует картинка местоположения
Для скважины 773.1 отсутствует картинка местоположения
Для скважи

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


save_pic 945.3
Для скважины 945.3 отсутствует картинка местоположения
save_pic 767.4
Для скважины 767.4 отсутствует картинка местоположения
save_pic 808.2
Для скважины 808.2 отсутствует картинка местоположения
save_pic 824.3
Для скважины 824.3 отсутствует картинка местоположения
Для скважины 828.3 отсутствует картинка местоположения
Для скважины 807.3 отсутствует картинка местоположения
save_pic 773.3
Для скважины 773.3 отсутствует картинка местоположения
save_pic 768.1
Для скважины 768.1 отсутствует картинка местоположения
Vse


# Ебучий хлам

In [110]:
        # Строим диаграмму
        
#         c1 = LineChart()
#         c1.title = "График температур"
#         c1.style = 13
#         c1.y_axis.title = 'Глубина, м'
#         c1.x_axis.title = 'Температура грунта, °С'

#         data = Reference(ws, min_col=5, min_row=10, max_col=18, max_row=16)
#         c1.add_data(data, titles_from_data=False)
        
#         ws.add_chart(c1, "A18")
    
        chart = ScatterChart()
        #стиль подписей
        chart.title = "График температур"
        chart.style = 2
        chart.y_axis.title = 'Температура грунта, °С'
        chart.x_axis.title = 'Глубина, м'


        # размер диаграммы
        chart.height = 15
        chart.width = 30
        chart.legend.position = 'b'
        # граничные области
#         chart.x_axis.scaling.min = 0
#         chart.y_axis.scaling.min = 0
#         chart.x_axis.scaling.max = 11
#         chart.y_axis.scaling.max = 1.5

        #диапазон таблицы
        minimum_col=5
        minimum_row=10
        maximum_col=18
        maximum_row=minimum_row+df_of_well.shape[0]
        
        xvalues = Reference(ws, min_row=9, min_col=minimum_col, max_col=maximum_col)

        for i in range(minimum_row, maximum_row):
            
            values = Reference(ws, min_row=i, min_col=minimum_col, max_col=maximum_col)
            series = Series(values, xvalues, title = ws.cell(column = 1, row = i).value)
            chart.series.append(series)
            s1 = chart.series[i-minimum_row]
            s1.marker.symbol = "circle"
#             s1.marker.graphicalProperties.solidFill = "FF0000" # Marker filling
#             s1.marker.graphicalProperties.line.solidFill = "FF0000" # Marker outline
#             s1.graphicalProperties.line.width = 25000
#             s1.graphicalProperties.line.solidFill = colour_list[i-minimum_row]
#             s1.graphicalProperties.line.noFill = True

        ws.add_chart(chart, "A25")

Help on bool object:

class bool(int)
 |  bool(x) -> bool
 |  
 |  Returns True when the argument x is true, False otherwise.
 |  The builtins True and False are the only two instances of the class bool.
 |  The class bool is a subclass of the class int, and cannot be subclassed.
 |  
 |  Method resolution order:
 |      bool
 |      int
 |      object
 |  
 |  Methods defined here:
 |  
 |  __and__(self, value, /)
 |      Return self&value.
 |  
 |  __or__(self, value, /)
 |      Return self|value.
 |  
 |  __rand__(self, value, /)
 |      Return value&self.
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  __ror__(self, value, /)
 |      Return value|self.
 |  
 |  __rxor__(self, value, /)
 |      Return value^self.
 |  
 |  __str__(self, /)
 |      Return str(self).
 |  
 |  __xor__(self, value, /)
 |      Return self^value.
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) 